### Loading Required Modules

In [1]:
"""
Loading Required Modules
"""
from pdfminer3.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer3.converter import TextConverter
from pdfminer3.layout import LAParams
from pdfminer3.pdfpage import PDFPage
from io import BytesIO
from io import StringIO
import os
import pandas as pd
import numpy as np

## Keyword Search Utility 
from flashtext import KeywordProcessor
import re
import string

from cleanco import cleanco
import re 
import string

import pdf_to_html as pth
import multiprocessing as mp

In [2]:
"""
Define The Paths
"""
RAW_DATA_PATH = "../2. Data/raw"
HTML_PATH = "../2. Data/html"
TXT_PATH = "../2. Data/txt"
PROT_DF_PATH =  "../2. Data/prot_df"

### Helper Functions

In [3]:
def convertPDFToText_String(path):
    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr,codec=codec,laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos=set()
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)
        fp.close()
        device.close()
        string = retstr.getvalue()
        retstr.close()
        return string
    except Exception as e:
        print(traceback.format_exc())
        print(e)
        return "Fail"

In [43]:
def clean_text_func(text):
        text = str(text)
        text = text.lower()
        text = re.sub(r'[^\x00-\x7F]+',' ', text)
        text = text.replace("\n", " ")
        translator = re.compile('[%s]' % re.escape(string.punctuation))
        text = translator.sub(' ', text)
        text = re.sub(' +', ' ', text)
        text = " ".join(text.split())
        text = text.strip()
        return text

class basic_keyword_extract:
    """Basic keyword extractor class using the keyword processor by Flashtext"""
    def __init__(self, keyword_list, case_sensitive=False):
        self.keyword_list = keyword_list
        self.case_sensitive = case_sensitive
        self.keyword_processor = self._initialize_basic_keyword_processor()
   
    def _initialize_basic_keyword_processor(self):
        kp = KeywordProcessor(case_sensitive=self.case_sensitive)
        kp.add_keywords_from_list(self.keyword_list)
        return kp
    def extract_from_text(self, raw_text):
        if self.case_sensitive:
            text = raw_text
        else:
            text = raw_text.lower().strip()
            clean_text = clean_text_func(text)
            
        clean_extracted_keywords_ls = self.keyword_processor.extract_keywords(clean_text, span_info=True)
#         output_ls = list(set([i[0] for i in clean_extracted_keywords_ls]))
        output_ls = [{'keyword':i[0], 'start_idx':i[1], 'end_idx':i[2]} for i in clean_extracted_keywords_ls]
#         output_ls = list({i['keyword']:i for i in reversed(output_ls)}.values()) ## Remove dups
        if len(output_ls) == 0:
            return []
        else:
            return output_ls

In [5]:
def clean_company_name(text):
    
    text = cleanco(text).clean_name()
    text = str(text)
    text = text.lower()
    text = text.lower().replace(' mfg. co. ltd.', ' ').replace('group', '').replace(
                ' & co. ltd.', ' ').replace(' co. ltd.', ' ').replace(' ltd.', ' ').replace('ltd', '').replace('corporation', '').replace(
                'limited', ' ').replace(' pvt.', ' ').replace('pvt', '').replace('company', ' ').replace('india', '').replace('private', '').strip()
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    text = translator.sub(' ', text)
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    text = text.strip()
    return text

In [6]:
def clean_special_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    text = translator.sub(' ', text)
    text = re.sub(r'(?<=\b[a-z]) (?=[a-z]\b)', '', text)
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    text = text.strip()
    return text

In [7]:
def clean_ner_text(text):
    text = str(text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    return text

### Experience

#### Company Extraction

In [8]:
## Company Names
file = "../2. Data/indian_companies_list_v5.txt"
file = open(file, "r", encoding='utf-8')

## Create Company List
COMPANIES_LIST = [line.strip().lower() for line in file]

In [9]:
## Initialize Keyword Processor:
COMPANY_KEYWORDS_SEARCH = basic_keyword_extract(COMPANIES_LIST, case_sensitive=False)

In [10]:
## Function to extract company names
def get_company_names(text):
    text = clean_special_text(text)
    company_ls = COMPANY_KEYWORDS_SEARCH.extract_from_text(text)
    return company_ls

In [11]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    company = get_company_names(text)
    print("Companies: ", company, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Companies:  [{'keyword': 'shah associates', 'start_idx': 8488, 'end_idx': 8503}, {'keyword': 'fine line circuits limited', 'start_idx': 8409, 'end_idx': 8435}, {'keyword': 'godrej group', 'start_idx': 5486, 'end_idx': 5498}, {'keyword': 'kpmg', 'start_idx': 3839, 'end_idx': 3843}, {'keyword': 'the hershey company', 'start_idx': 5525, 'end_idx': 5544}, {'keyword': 'godrej hershey', 'start_idx': 5363, 'end_idx': 5377}, {'keyword': 'ambit pragma advisors llp', 'start_idx': 1002, 'end_idx': 1027}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Companies:  [{'keyword': 'arthur andersen', 'start_idx': 4766, 'end_idx': 4781}, {'keyword': 'bnp paribas', 'start_idx': 218, 'end_idx': 229}, {'keyword': 'ankur seeds', 'start_idx': 4307, 'end_idx': 4318}, {'keyword': 'icici bank', 'start_idx': 230, 'end_idx': 240}, {'keyword': 'religare finvest', 'start_idx': 245, 'end_idx': 261}]
---------------------------------------------- 
File

Companies:  []
---------------------------------------------- 
FileName :  ANSHUL AGARWAL- CV.pdf
Companies:  [{'keyword': 'ernst young', 'start_idx': 2032, 'end_idx': 2043}, {'keyword': 'karvy stock broking limited', 'start_idx': 1574, 'end_idx': 1601}, {'keyword': 'hexaware technologies', 'start_idx': 575, 'end_idx': 596}]
---------------------------------------------- 
FileName :  ANSHUL KUMAR GUPTA- CV (Rinalytics Advisors).pdf
Companies:  [{'keyword': 'uninstall io', 'start_idx': 271, 'end_idx': 283}]
---------------------------------------------- 
FileName :  Anshuman Kumar Rath- CV.pdf
Companies:  [{'keyword': 'aon hewitt', 'start_idx': 3880, 'end_idx': 3890}, {'keyword': 'bank muscat', 'start_idx': 861, 'end_idx': 872}, {'keyword': 'hdfc bank', 'start_idx': 1580, 'end_idx': 1589}, {'keyword': 'network 18', 'start_idx': 1519, 'end_idx': 1529}, {'keyword': 'tata motors', 'start_idx': 2083, 'end_idx': 2094}]
---------------------------------------------- 
FileName :  Anshuman Mish

Companies:  [{'keyword': 'torrent pharmaceuticals', 'start_idx': 720, 'end_idx': 743}, {'keyword': 'ajanta pharma', 'start_idx': 1957, 'end_idx': 1970}, {'keyword': 'piramal critical care', 'start_idx': 1805, 'end_idx': 1826}]
---------------------------------------------- 
FileName :  Ashwin N- CV.pdf
Companies:  [{'keyword': 'hsbc', 'start_idx': 1764, 'end_idx': 1768}]
---------------------------------------------- 
FileName :  AVIDHA CHAKRABARTI- CV.pdf
Companies:  [{'keyword': 'icici bank', 'start_idx': 3240, 'end_idx': 3250}, {'keyword': 'morgan stanley', 'start_idx': 219, 'end_idx': 233}]
---------------------------------------------- 
FileName :  Avinash Shekhar- CV.pdf
Companies:  [{'keyword': 'bharti airtel limited', 'start_idx': 526, 'end_idx': 547}, {'keyword': 'bharti enterprises', 'start_idx': 4373, 'end_idx': 4391}, {'keyword': 'future group', 'start_idx': 2457, 'end_idx': 2469}, {'keyword': 'altais advisors private limited', 'start_idx': 3373, 'end_idx': 3404}, {'keyword

Companies:  []
---------------------------------------------- 
FileName :  Dilip Menda- CV (Ishwa).pdf
Companies:  [{'keyword': 'reliance retail', 'start_idx': 6674, 'end_idx': 6689}, {'keyword': 'johnson johnson', 'start_idx': 30, 'end_idx': 45}]
---------------------------------------------- 
FileName :  Dinesh Chhabra- CV.pdf
Companies:  [{'keyword': 'eicher limited', 'start_idx': 1846, 'end_idx': 1860}, {'keyword': 'force motors', 'start_idx': 1530, 'end_idx': 1542}, {'keyword': 'dinesh co', 'start_idx': 3077, 'end_idx': 3086}, {'keyword': 'usha international', 'start_idx': 42, 'end_idx': 60}, {'keyword': 'thirdware solutions', 'start_idx': 1700, 'end_idx': 1719}]
---------------------------------------------- 
FileName :  DIPESH MOHILE- CV.pdf
Companies:  [{'keyword': 'cisco systems', 'start_idx': 1807, 'end_idx': 1820}, {'keyword': 'tonse telecom', 'start_idx': 1556, 'end_idx': 1569}]
---------------------------------------------- 
FileName :  Dipty Jain- CV.pdf
Companies:  [{'ke

Companies:  [{'keyword': 'mcdonald s', 'start_idx': 4066, 'end_idx': 4076}, {'keyword': 'yum restaurants india', 'start_idx': 2567, 'end_idx': 2588}, {'keyword': 'pizza hut', 'start_idx': 1172, 'end_idx': 1181}]
---------------------------------------------- 
FileName :  GIBIN S KOLLAMPARAMBIL- CV.pdf
Companies:  [{'keyword': 'axience consulting', 'start_idx': 1630, 'end_idx': 1648}, {'keyword': 'texmex cuisine', 'start_idx': 688, 'end_idx': 702}, {'keyword': 'venture intelligence', 'start_idx': 333, 'end_idx': 353}]
---------------------------------------------- 
FileName :  Giya Diwaan- CV (ISHWA).pdf
Companies:  [{'keyword': 'idbi bank limited', 'start_idx': 11052, 'end_idx': 11069}, {'keyword': 'moody s analytics', 'start_idx': 9450, 'end_idx': 9467}, {'keyword': 'thomas cook', 'start_idx': 6915, 'end_idx': 6926}, {'keyword': 'canara bank', 'start_idx': 6538, 'end_idx': 6549}, {'keyword': 'essel group', 'start_idx': 5786, 'end_idx': 5797}, {'keyword': 'itzcash card', 'start_idx': 5

...

Companies:  []
---------------------------------------------- 
FileName :  Roopam Upadhyay- CV.pdf
Companies:  [{'keyword': 'major brands india private limited', 'start_idx': 7374, 'end_idx': 7408}, {'keyword': 'wipro infotech', 'start_idx': 3369, 'end_idx': 3383}, {'keyword': 'care kalypto', 'start_idx': 5991, 'end_idx': 6003}, {'keyword': 'kotak securities', 'start_idx': 4999, 'end_idx': 5015}, {'keyword': 'axis bank', 'start_idx': 690, 'end_idx': 699}, {'keyword': 'credit risk management', 'start_idx': 1378, 'end_idx': 1400}, {'keyword': 'future group', 'start_idx': 1200, 'end_idx': 1212}, {'keyword': 'hapag lloyd', 'start_idx': 1075, 'end_idx': 1086}, {'keyword': 'crompton greaves', 'start_idx': 985, 'end_idx': 1001}, {'keyword': 'john deere', 'start_idx': 974, 'end_idx': 984}, {'keyword': 'icici bank', 'start_idx': 920, 'end_idx': 930}, {'keyword': 'mahindra mahindra', 'start_idx': 705, 'end_idx': 722}, {'keyword': 'reliance capital', 'start_idx': 673, 'end_idx': 689}]
-----------

Companies:  [{'keyword': 'ge capital', 'start_idx': 6655, 'end_idx': 6665}, {'keyword': 'genpact india', 'start_idx': 6578, 'end_idx': 6591}, {'keyword': 'tata iq', 'start_idx': 269, 'end_idx': 276}, {'keyword': 'tata steel', 'start_idx': 853, 'end_idx': 863}, {'keyword': 'active marketing', 'start_idx': 478, 'end_idx': 494}, {'keyword': 'tata group', 'start_idx': 422, 'end_idx': 432}]
---------------------------------------------- 
FileName :  Saurabh Gupta- CV.pdf
Companies:  [{'keyword': 'schneider electric', 'start_idx': 13621, 'end_idx': 13639}, {'keyword': 'rockland hospital', 'start_idx': 2123, 'end_idx': 2140}, {'keyword': 'ge healthcare', 'start_idx': 1825, 'end_idx': 1838}, {'keyword': 'tata steel', 'start_idx': 2993, 'end_idx': 3003}, {'keyword': 'fortis healthcare', 'start_idx': 1615, 'end_idx': 1632}, {'keyword': 'sl raheja hospital', 'start_idx': 1596, 'end_idx': 1614}]
---------------------------------------------- 
FileName :  Saurabh SARKAR- CV.pdf
Companies:  [{'keywo

Companies:  [{'keyword': 'state bank of india', 'start_idx': 6742, 'end_idx': 6761}, {'keyword': 'fortis healthcare limited', 'start_idx': 3414, 'end_idx': 3439}, {'keyword': 'fortis healthcare', 'start_idx': 3342, 'end_idx': 3359}, {'keyword': 'shrachi group', 'start_idx': 520, 'end_idx': 533}]
---------------------------------------------- 
FileName :  Sonal Varma- CV.pdf
Companies:  [{'keyword': 'icici bank', 'start_idx': 246, 'end_idx': 256}, {'keyword': 'lehman brothers', 'start_idx': 172, 'end_idx': 187}]
---------------------------------------------- 
FileName :  Sonali Gupta- CV.pdf
Companies:  [{'keyword': 'ericsson india private limited', 'start_idx': 1066, 'end_idx': 1096}]
---------------------------------------------- 
FileName :  Sreedhar Reddy M- CV.pdf
Companies:  []
---------------------------------------------- 
FileName :  Srikanth Nadhamuni- CV.pdf
Companies:  [{'keyword': 'hdfc bank', 'start_idx': 35, 'end_idx': 44}, {'keyword': 'sun microsystems', 'start_idx': 226

Companies:  [{'keyword': 'ernst young llp', 'start_idx': 1150, 'end_idx': 1165}, {'keyword': 'kotak investment banking', 'start_idx': 73, 'end_idx': 97}]
---------------------------------------------- 
FileName :  Usha Prabhakar- CV (ISHWA).pdf
Companies:  [{'keyword': 'hotel crown', 'start_idx': 6497, 'end_idx': 6508}, {'keyword': 'the pioneer', 'start_idx': 5569, 'end_idx': 5580}, {'keyword': 'healthcare at home', 'start_idx': 653, 'end_idx': 671}, {'keyword': 'dhrc hospital', 'start_idx': 2659, 'end_idx': 2672}, {'keyword': 'rockland hospital', 'start_idx': 1810, 'end_idx': 1827}, {'keyword': 'king george hospital', 'start_idx': 1735, 'end_idx': 1755}]
---------------------------------------------- 
FileName :  V Venkatasubramanian- CV (ISHWA).pdf
Companies:  [{'keyword': 'apollo hospitals', 'start_idx': 3371, 'end_idx': 3387}, {'keyword': 'aravind eye care', 'start_idx': 8673, 'end_idx': 8689}, {'keyword': 'sankara nethralaya', 'start_idx': 3054, 'end_idx': 3072}, {'keyword': 'kast

Companies:  [{'keyword': 'hinduja global solutions', 'start_idx': 4306, 'end_idx': 4330}, {'keyword': 'hinduja group', 'start_idx': 4040, 'end_idx': 4053}, {'keyword': 'adm services', 'start_idx': 1642, 'end_idx': 1654}, {'keyword': 'gss infotech limited', 'start_idx': 1539, 'end_idx': 1559}, {'keyword': 'mahesh tutorials', 'start_idx': 667, 'end_idx': 683}, {'keyword': 'mt educare', 'start_idx': 645, 'end_idx': 655}]
---------------------------------------------- 
FileName :  Yash Bansal- CV.pdf
Companies:  [{'keyword': 'grant thornton', 'start_idx': 1709, 'end_idx': 1723}, {'keyword': 'mape advisory group', 'start_idx': 106, 'end_idx': 125}]
---------------------------------------------- 
FileName :  Yatin Joshi- CV.pdf
Companies:  [{'keyword': 'arabi company wll', 'start_idx': 7442, 'end_idx': 7459}, {'keyword': 'rdc concrete india', 'start_idx': 4195, 'end_idx': 4213}, {'keyword': 'fosroc chemicals india', 'start_idx': 2922, 'end_idx': 2944}, {'keyword': 'ambuja cements', 'start_id

In [12]:
# company_ls = list(set(company_ls) - set(remove_ls))

# with open('../2. Data/indian_companies_list_v6.txt', 'w', encoding='utf-8') as f:
#     for item in company_ls:
#         f.write("%s\n" % item)

#### Designation Extraction

In [11]:
## Company Names
file = "../2. Data/job_titles_list_v2.txt"
file = open(file, "r", encoding='utf-8')

TITLE_LIST = [line.strip().lower() for line in file]

In [12]:
## Initialize Keyword Processor:
TITLE_KEYWORDS_SEARCH = basic_keyword_extract(TITLE_LIST, case_sensitive=False)

In [13]:
def get_job_titles(text):
    text = clean_special_text(text)
    title_ls = TITLE_KEYWORDS_SEARCH.extract_from_text(text)
    return title_ls

In [16]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    designation = get_job_titles(text)
    print("Designation: ", designation, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Designation:  [{'keyword': 'vice president', 'start_idx': 6515, 'end_idx': 6529}, {'keyword': 'assistant manager', 'start_idx': 5624, 'end_idx': 5641}, {'keyword': 'senior consultant', 'start_idx': 423, 'end_idx': 440}, {'keyword': 'project manager', 'start_idx': 2711, 'end_idx': 2726}, {'keyword': 'finance professional', 'start_idx': 59, 'end_idx': 79}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Designation:  [{'keyword': 'consultant', 'start_idx': 4325, 'end_idx': 4335}, {'keyword': 'credit manager', 'start_idx': 858, 'end_idx': 872}, {'keyword': 'chartered accountant', 'start_idx': 122, 'end_idx': 142}]
---------------------------------------------- 
FileName :  Abhilash Pillai- CV.pdf
Designation:  [{'keyword': 'director', 'start_idx': 16, 'end_idx': 24}, {'keyword': 'senior software developer', 'start_idx': 5256, 'end_idx': 5281}, {'keyword': 'sales marketing', 'start_idx': 4829, 'end_idx': 4844}, {'keyword': '

Designation:  [{'keyword': 'operations analyst', 'start_idx': 7618, 'end_idx': 7636}, {'keyword': 'management trainee', 'start_idx': 7568, 'end_idx': 7586}, {'keyword': 'senior analyst', 'start_idx': 5583, 'end_idx': 5597}, {'keyword': 'securities analyst', 'start_idx': 6050, 'end_idx': 6068}, {'keyword': 'associate', 'start_idx': 4561, 'end_idx': 4570}, {'keyword': 'operator', 'start_idx': 4153, 'end_idx': 4161}, {'keyword': 'cio', 'start_idx': 2352, 'end_idx': 2355}, {'keyword': 'portfolio manager', 'start_idx': 2183, 'end_idx': 2200}, {'keyword': 'founder', 'start_idx': 1806, 'end_idx': 1813}, {'keyword': 'chairman', 'start_idx': 1793, 'end_idx': 1801}, {'keyword': 'senior vice president', 'start_idx': 119, 'end_idx': 140}]
---------------------------------------------- 
FileName :  Amit Singh- CV.pdf
Designation:  [{'keyword': 'legal counsel', 'start_idx': 39, 'end_idx': 52}, {'keyword': 'legal officer', 'start_idx': 3998, 'end_idx': 4011}, {'keyword': 'manager legal', 'start_idx':

Designation:  [{'keyword': 'blog writer', 'start_idx': 1817, 'end_idx': 1828}, {'keyword': 'research intern', 'start_idx': 1256, 'end_idx': 1271}, {'keyword': 'science intern', 'start_idx': 1128, 'end_idx': 1142}, {'keyword': 'data scientist', 'start_idx': 256, 'end_idx': 270}]
---------------------------------------------- 
FileName :  Anshuman Kumar Rath- CV.pdf
Designation:  [{'keyword': 'hr business partner', 'start_idx': 5203, 'end_idx': 5222}, {'keyword': 'process owner', 'start_idx': 3793, 'end_idx': 3806}, {'keyword': 'project manager', 'start_idx': 3777, 'end_idx': 3792}, {'keyword': 'graduate engineer', 'start_idx': 1907, 'end_idx': 1924}, {'keyword': 'assistant manager', 'start_idx': 1886, 'end_idx': 1903}, {'keyword': 'hr manager', 'start_idx': 1739, 'end_idx': 1749}, {'keyword': 'senior consultant', 'start_idx': 1721, 'end_idx': 1738}, {'keyword': 'vice president', 'start_idx': 1395, 'end_idx': 1409}, {'keyword': 'human resources professional', 'start_idx': 275, 'end_idx':

Designation:  [{'keyword': 'sr manager', 'start_idx': 7923, 'end_idx': 7933}, {'keyword': 'senior manager', 'start_idx': 7870, 'end_idx': 7884}, {'keyword': 'management trainee', 'start_idx': 7467, 'end_idx': 7485}, {'keyword': 'hr business partner', 'start_idx': 4483, 'end_idx': 4502}, {'keyword': 'sales operations', 'start_idx': 6613, 'end_idx': 6629}, {'keyword': 'regional hr manager', 'start_idx': 1487, 'end_idx': 1506}, {'keyword': 'human resources manager', 'start_idx': 5962, 'end_idx': 5985}, {'keyword': 'practice managers', 'start_idx': 5513, 'end_idx': 5530}, {'keyword': 'process owner', 'start_idx': 4455, 'end_idx': 4468}, {'keyword': 'director', 'start_idx': 2201, 'end_idx': 2209}, {'keyword': 'research manager', 'start_idx': 4147, 'end_idx': 4163}, {'keyword': 'project leader', 'start_idx': 3204, 'end_idx': 3218}, {'keyword': 'executive director', 'start_idx': 3130, 'end_idx': 3148}, {'keyword': 'associate director', 'start_idx': 1259, 'end_idx': 1277}, {'keyword': 'ceo', '

Designation:  [{'keyword': 'advisor consultant', 'start_idx': 4019, 'end_idx': 4037}, {'keyword': 'co founder', 'start_idx': 564, 'end_idx': 574}, {'keyword': 'vp', 'start_idx': 3412, 'end_idx': 3414}, {'keyword': 'managing director', 'start_idx': 2705, 'end_idx': 2722}, {'keyword': 'director', 'start_idx': 36, 'end_idx': 44}, {'keyword': 'vice president', 'start_idx': 1118, 'end_idx': 1132}, {'keyword': 'senior product manager', 'start_idx': 935, 'end_idx': 957}, {'keyword': 'owner', 'start_idx': 871, 'end_idx': 876}, {'keyword': 'avp', 'start_idx': 775, 'end_idx': 778}, {'keyword': 'product development', 'start_idx': 755, 'end_idx': 774}, {'keyword': 'consultant', 'start_idx': 506, 'end_idx': 516}]
---------------------------------------------- 
FileName :  BARNALI ROYCHOWDHURY- CV.pdf
Designation:  [{'keyword': 'management trainee', 'start_idx': 6376, 'end_idx': 6394}, {'keyword': 'ceo', 'start_idx': 743, 'end_idx': 746}, {'keyword': 'recruitment specialist', 'start_idx': 2511, 'end

...

Designation:  [{'keyword': 'owner', 'start_idx': 1422, 'end_idx': 1427}, {'keyword': 'director', 'start_idx': 1240, 'end_idx': 1248}, {'keyword': 'management consultant', 'start_idx': 197, 'end_idx': 218}]
---------------------------------------------- 
FileName :  Shriram Krishnan- CV.pdf
Designation:  [{'keyword': 'finance professional', 'start_idx': 19, 'end_idx': 39}, {'keyword': 'counsel', 'start_idx': 425, 'end_idx': 432}, {'keyword': 'vp', 'start_idx': 253, 'end_idx': 255}, {'keyword': 'cfo', 'start_idx': 123, 'end_idx': 126}, {'keyword': 'consultant', 'start_idx': 59, 'end_idx': 69}]
---------------------------------------------- 
FileName :  SHUBHANGA PRASAD- CV.pdf
Designation:  [{'keyword': 'team lead', 'start_idx': 3853, 'end_idx': 3862}, {'keyword': 'faculty member', 'start_idx': 3020, 'end_idx': 3034}, {'keyword': 'product development', 'start_idx': 2652, 'end_idx': 2671}, {'keyword': 'partner', 'start_idx': 1458, 'end_idx': 1465}, {'keyword': 'digital tech', 'start_idx':

Designation:  [{'keyword': 'certified public accountant', 'start_idx': 2312, 'end_idx': 2339}, {'keyword': 'company secretary', 'start_idx': 1552, 'end_idx': 1569}, {'keyword': 'chartered accountant', 'start_idx': 2075, 'end_idx': 2095}, {'keyword': 'management trainee', 'start_idx': 1956, 'end_idx': 1974}, {'keyword': 'finance executive', 'start_idx': 1884, 'end_idx': 1901}, {'keyword': 'avp', 'start_idx': 1697, 'end_idx': 1700}, {'keyword': 'vp', 'start_idx': 1694, 'end_idx': 1696}, {'keyword': 'financial controller', 'start_idx': 1531, 'end_idx': 1551}, {'keyword': 'cfo', 'start_idx': 1461, 'end_idx': 1464}, {'keyword': 'finance professional', 'start_idx': 139, 'end_idx': 159}]
---------------------------------------------- 
FileName :  Sujan Hajra- CV.pdf
Designation:  [{'keyword': 'executive director', 'start_idx': 12, 'end_idx': 30}, {'keyword': 'director', 'start_idx': 1923, 'end_idx': 1931}, {'keyword': 'vp', 'start_idx': 1867, 'end_idx': 1869}, {'keyword': 'senior vice preside

Designation:  [{'keyword': 'company secretary', 'start_idx': 6503, 'end_idx': 6520}, {'keyword': 'cfo', 'start_idx': 5326, 'end_idx': 5329}, {'keyword': 'manager financial planning', 'start_idx': 873, 'end_idx': 899}, {'keyword': 'business advisor', 'start_idx': 1448, 'end_idx': 1464}, {'keyword': 'financial controller', 'start_idx': 826, 'end_idx': 846}]
---------------------------------------------- 
FileName :  Tejash Mahendra Tanna- CV.pdf
Designation:  [{'keyword': 'manager internal', 'start_idx': 4431, 'end_idx': 4447}, {'keyword': 'cfo', 'start_idx': 369, 'end_idx': 372}, {'keyword': 'chief financial officer', 'start_idx': 345, 'end_idx': 368}, {'keyword': 'chartered accountant', 'start_idx': 22, 'end_idx': 42}]
---------------------------------------------- 
FileName :  Uddeeptabh Mishra- CV (Kelly Services).pdf
Designation:  [{'keyword': 'learning designer', 'start_idx': 5006, 'end_idx': 5023}, {'keyword': 'process owner', 'start_idx': 3262, 'end_idx': 3275}, {'keyword': 'mana

Designation:  [{'keyword': 'regional director', 'start_idx': 22654, 'end_idx': 22671}, {'keyword': 'chief financial officer', 'start_idx': 20774, 'end_idx': 20797}, {'keyword': 'financial controller', 'start_idx': 3563, 'end_idx': 3583}, {'keyword': 'deputy manager', 'start_idx': 3303, 'end_idx': 3317}, {'keyword': 'pricing coordinator', 'start_idx': 19909, 'end_idx': 19928}, {'keyword': 'site controller', 'start_idx': 3010, 'end_idx': 3025}, {'keyword': 'support director', 'start_idx': 15591, 'end_idx': 15607}, {'keyword': 'partner', 'start_idx': 15228, 'end_idx': 15235}, {'keyword': 'business controller', 'start_idx': 2956, 'end_idx': 2975}, {'keyword': 'director', 'start_idx': 539, 'end_idx': 547}, {'keyword': 'sales operations', 'start_idx': 11388, 'end_idx': 11404}, {'keyword': 'cfo', 'start_idx': 565, 'end_idx': 568}, {'keyword': 'site director', 'start_idx': 7709, 'end_idx': 7722}, {'keyword': 'co pilot', 'start_idx': 3851, 'end_idx': 3859}, {'keyword': 'chartered accountant', '

Designation:  [{'keyword': 'tax consultant', 'start_idx': 1957, 'end_idx': 1971}]
---------------------------------------------- 
FileName :  Yatin Joshi- CV.pdf
Designation:  [{'keyword': 'sales manager', 'start_idx': 4946, 'end_idx': 4959}, {'keyword': 'business manager', 'start_idx': 6981, 'end_idx': 6997}, {'keyword': 'technical manager', 'start_idx': 5996, 'end_idx': 6013}, {'keyword': 'sales and marketing manager', 'start_idx': 5705, 'end_idx': 5732}, {'keyword': 'area sales manager', 'start_idx': 5540, 'end_idx': 5558}, {'keyword': 'product development', 'start_idx': 3398, 'end_idx': 3417}, {'keyword': 'national sales manager', 'start_idx': 4172, 'end_idx': 4194}, {'keyword': 'business development manager', 'start_idx': 3723, 'end_idx': 3751}, {'keyword': 'sales marketing', 'start_idx': 96, 'end_idx': 111}, {'keyword': 'area manager', 'start_idx': 2906, 'end_idx': 2918}, {'keyword': 'sales marketing manager', 'start_idx': 2371, 'end_idx': 2394}]
---------------------------------

In [17]:
# title_ls = list(set(title_ls) - set(remove_ls))

# with open('../2. Data/job_titles_list_v2.txt', 'w', encoding='utf-8') as f:
#     for item in title_ls:
#         f.write("%s\n" % item)

#### Time Period Extraction

In [14]:
import spacy
import datetime
nlp = spacy.load('en_core_web_sm')

In [15]:
def get_period(text):
    text = clean_ner_text(text)
    doc = nlp(text)
    date_ls = []
    if doc.ents: 
        for ent in doc.ents: 
            date_dict = {'period':'', 'start_char':'', 'end_char':''}
            if ent.label_ == "DATE":
                date_dict['period'] = ent.text
                date_dict['start_char'] = ent.start_char
                date_dict['end_char'] = ent.end_char
                if len(DATE_KEYWORDS_SEARCH.extract_from_text(ent.text)) > 0:
                    date_ls.append(date_dict)
    return date_ls

In [16]:
start_year = 1947
end_year = int(datetime.datetime.now().year)
year_ls = [str(i) for i in np.arange(start_year, end_year)]

# DATE_KEYWORDS_LS = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec',
#                     'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november',
#                     'december',
#                     'date', 'present'] + year_ls

DATE_KEYWORDS_LS = year_ls

In [17]:
## Initialize Keyword Processor:
DATE_KEYWORDS_SEARCH = basic_keyword_extract(DATE_KEYWORDS_LS, case_sensitive=False)

In [83]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    tenure_ls = get_period(text)
    print("Time Period: ", tenure_ls, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Time Period:  [{'period': 'Nov 2012', 'start_char': 4198, 'end_char': 4206}, {'period': '2013 Monthly', 'start_char': 4211, 'end_char': 4223}, {'period': 'May 2009', 'start_char': 7605, 'end_char': 7613}, {'period': 'May 2004', 'start_char': 7620, 'end_char': 7628}, {'period': 'May 2004', 'start_char': 8726, 'end_char': 8734}, {'period': '2005', 'start_char': 9081, 'end_char': 9085}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Time Period:  [{'period': 'April 2017- February 2018', 'start_char': 726, 'end_char': 751}, {'period': 'July 2015 April 2017', 'start_char': 1033, 'end_char': 1053}, {'period': 'April 2011 - October 2014', 'start_char': 1975, 'end_char': 2000}, {'period': 'October 2003-March', 'start_char': 3244, 'end_char': 3262}, {'period': 'the years of 2008', 'start_char': 4114, 'end_char': 4131}, {'period': '2009', 'start_char': 4136, 'end_char': 4140}, {'period': 'September 2002 - October 2003', 'start_ch

Time Period:  [{'period': 'May, 2004', 'start_char': 550, 'end_char': 559}, {'period': '2012', 'start_char': 629, 'end_char': 633}, {'period': '2015', 'start_char': 842, 'end_char': 846}, {'period': 'Oct 2015', 'start_char': 941, 'end_char': 949}, {'period': 'May 2011', 'start_char': 1309, 'end_char': 1317}, {'period': 'Oct 2015', 'start_char': 1757, 'end_char': 1765}]
---------------------------------------------- 
FileName :  Aditya Narayan Shetty- CV.pdf
Time Period:  [{'period': 'August 2011', 'start_char': 757, 'end_char': 768}, {'period': 'August 2011', 'start_char': 1027, 'end_char': 1038}, {'period': 'November 2009 to July 2011', 'start_char': 2831, 'end_char': 2857}, {'period': 'May 2012', 'start_char': 4413, 'end_char': 4421}, {'period': '[2002 to 2005', 'start_char': 4778, 'end_char': 4791}, {'period': '2000 to 2002', 'start_char': 4900, 'end_char': 4912}, {'period': 'December 2003', 'start_char': 5117, 'end_char': 5130}, {'period': 'February 27, 1985', 'start_char': 5366, '

Time Period:  [{'period': '2012-13', 'start_char': 5715, 'end_char': 5722}, {'period': '2012-13', 'start_char': 6044, 'end_char': 6051}, {'period': '2012-13', 'start_char': 6231, 'end_char': 6238}, {'period': 'between 2012-13 & 2013-14', 'start_char': 6581, 'end_char': 6606}, {'period': '2012-13', 'start_char': 6666, 'end_char': 6673}, {'period': '2012-13', 'start_char': 6713, 'end_char': 6720}, {'period': '1998', 'start_char': 7807, 'end_char': 7811}, {'period': '1994', 'start_char': 7890, 'end_char': 7894}]
---------------------------------------------- 
FileName :  Anchal Sultania- CV.pdf
Time Period:  [{'period': '2001', 'start_char': 10987, 'end_char': 10991}, {'period': 'March 2010', 'start_char': 11570, 'end_char': 11580}, {'period': 'Sept Dec 2009 July 2005', 'start_char': 11581, 'end_char': 11604}]
---------------------------------------------- 
FileName :  Andrew Stansbie- CV (Hospitale Recruitment).pdf
Time Period:  [{'period': 'August 2007', 'start_char': 328, 'end_char': 3

Time Period:  [{'period': 'August 2013', 'start_char': 514, 'end_char': 525}, {'period': 'December 2014', 'start_char': 828, 'end_char': 841}, {'period': 'May 2016', 'start_char': 1024, 'end_char': 1032}, {'period': 'May / June 2017', 'start_char': 1786, 'end_char': 1801}, {'period': '2008', 'start_char': 2316, 'end_char': 2320}, {'period': '2009', 'start_char': 2325, 'end_char': 2329}]
---------------------------------------------- 
FileName :  ANSHUL KUMAR GUPTA- CV (Rinalytics Advisors).pdf
Time Period:  [{'period': '2012-2016', 'start_char': 80, 'end_char': 89}, {'period': '2011 2009', 'start_char': 121, 'end_char': 130}, {'period': '2013', 'start_char': 1965, 'end_char': 1969}]
---------------------------------------------- 
FileName :  Anshuman Kumar Rath- CV.pdf
Time Period:  [{'period': 'May 2014', 'start_char': 1603, 'end_char': 1611}, {'period': 'April 2015', 'start_char': 1624, 'end_char': 1634}, {'period': 'April 2007', 'start_char': 1691, 'end_char': 1701}, {'period': 'May

Time Period:  [{'period': '2015', 'start_char': 81, 'end_char': 85}, {'period': '2012', 'start_char': 2625, 'end_char': 2629}, {'period': 'July 2009 to March 2012', 'start_char': 5221, 'end_char': 5244}, {'period': '2004', 'start_char': 7468, 'end_char': 7472}, {'period': 'Jan 2008', 'start_char': 7476, 'end_char': 7484}, {'period': 'May 2002', 'start_char': 8477, 'end_char': 8485}, {'period': 'June 1998', 'start_char': 8768, 'end_char': 8777}, {'period': 'May 2002', 'start_char': 8781, 'end_char': 8789}, {'period': '1995', 'start_char': 8837, 'end_char': 8841}, {'period': 'May 1998', 'start_char': 9016, 'end_char': 9024}, {'period': '1992', 'start_char': 9139, 'end_char': 9143}, {'period': '1990', 'start_char': 9203, 'end_char': 9207}, {'period': '25TH August 1974', 'start_char': 9234, 'end_char': 9250}]
---------------------------------------------- 
FileName :  ARNOV KUMAR- CV.pdf
Time Period:  [{'period': 'April 2011', 'start_char': 279, 'end_char': 289}, {'period': 'December 2015'

...

Time Period:  [{'period': 'during1989-1990', 'start_char': 11089, 'end_char': 11104}]
---------------------------------------------- 
FileName :  Srikanth Nadhamuni- CV.pdf
Time Period:  [{'period': '2003', 'start_char': 814, 'end_char': 818}, {'period': 'September 2016 - Present', 'start_char': 1357, 'end_char': 1381}, {'period': 'October 2013', 'start_char': 1452, 'end_char': 1464}, {'period': 'October 2012', 'start_char': 1526, 'end_char': 1538}, {'period': 'July 2009 - September 2012', 'start_char': 1619, 'end_char': 1645}, {'period': 'March 2003 - June 2009', 'start_char': 1939, 'end_char': 1961}, {'period': 'January 2000', 'start_char': 2014, 'end_char': 2026}, {'period': 'February 1996', 'start_char': 2097, 'end_char': 2110}, {'period': 'July 1994 - February 1996', 'start_char': 2186, 'end_char': 2211}, {'period': 'February 1993 - March 1994', 'start_char': 2266, 'end_char': 2292}, {'period': 'January 1989 - February 1993', 'start_char': 2356, 'end_char': 2384}, {'period': '1986

Time Period:  [{'period': 'Dec 2013', 'start_char': 1285, 'end_char': 1293}, {'period': 'Jan 2011 Jan 2013', 'start_char': 2547, 'end_char': 2564}, {'period': '2011', 'start_char': 3105, 'end_char': 3109}, {'period': '2011', 'start_char': 3147, 'end_char': 3151}, {'period': '2012', 'start_char': 3230, 'end_char': 3234}, {'period': '2012', 'start_char': 3319, 'end_char': 3323}, {'period': 'Dec 2010', 'start_char': 3333, 'end_char': 3341}, {'period': 'May 2000', 'start_char': 4854, 'end_char': 4862}, {'period': 'Oct 2004', 'start_char': 4863, 'end_char': 4871}, {'period': '2000', 'start_char': 7278, 'end_char': 7282}, {'period': '1999', 'start_char': 7546, 'end_char': 7550}, {'period': '1997', 'start_char': 9025, 'end_char': 9029}, {'period': '1996', 'start_char': 9198, 'end_char': 9202}, {'period': '2010', 'start_char': 9354, 'end_char': 9358}, {'period': '2009', 'start_char': 9422, 'end_char': 9426}, {'period': '2007', 'start_char': 9534, 'end_char': 9538}, {'period': '2015', 'start_ch

Time Period:  [{'period': '2016 June 2017', 'start_char': 553, 'end_char': 567}, {'period': 'July 2014', 'start_char': 2794, 'end_char': 2803}, {'period': '2016', 'start_char': 3615, 'end_char': 3619}, {'period': '2015', 'start_char': 3802, 'end_char': 3806}, {'period': 'July 2013', 'start_char': 4046, 'end_char': 4055}]
---------------------------------------------- 
FileName :  Usha Prabhakar- CV (ISHWA).pdf
Time Period:  [{'period': '2006 2011', 'start_char': 821, 'end_char': 830}, {'period': '2012', 'start_char': 1302, 'end_char': 1306}, {'period': 'Jan 2006', 'start_char': 2605, 'end_char': 2613}, {'period': 'July-Dec 2012', 'start_char': 2764, 'end_char': 2777}, {'period': 'June 1995', 'start_char': 4614, 'end_char': 4623}, {'period': '2002', 'start_char': 4670, 'end_char': 4674}, {'period': '2002', 'start_char': 4745, 'end_char': 4749}, {'period': '2006', 'start_char': 4789, 'end_char': 4793}, {'period': '2013 M. Phil', 'start_char': 4840, 'end_char': 4852}, {'period': '12th Apr

Time Period:  [{'period': 'October 2007 - December 2009', 'start_char': 597, 'end_char': 625}, {'period': 'March 2007 - October 2007', 'start_char': 706, 'end_char': 731}, {'period': 'June 2005 - March 2007', 'start_char': 789, 'end_char': 811}, {'period': '2000 - 2005', 'start_char': 867, 'end_char': 878}, {'period': '1999 - 2000', 'start_char': 939, 'end_char': 950}, {'period': '1996 - 1999', 'start_char': 1010, 'end_char': 1021}, {'period': '1994 - 1997', 'start_char': 1062, 'end_char': 1073}]
---------------------------------------------- 
FileName :  Vikas Mittal- CV.pdf
Time Period:  [{'period': '2012', 'start_char': 955, 'end_char': 959}, {'period': '2017', 'start_char': 986, 'end_char': 990}, {'period': '2001', 'start_char': 1421, 'end_char': 1425}, {'period': 'the year 2011', 'start_char': 1531, 'end_char': 1544}, {'period': 'February 1977', 'start_char': 1927, 'end_char': 1940}, {'period': '2007', 'start_char': 2254, 'end_char': 2258}, {'period': 'July 2012', 'start_char': 28

Time Period:  [{'period': 'August 2015', 'start_char': 340, 'end_char': 351}, {'period': '2014', 'start_char': 483, 'end_char': 487}, {'period': 'August 2015', 'start_char': 533, 'end_char': 544}, {'period': 'August 2015', 'start_char': 617, 'end_char': 628}, {'period': 'July 2015', 'start_char': 965, 'end_char': 974}, {'period': 'August 2014 to July 2015', 'start_char': 1047, 'end_char': 1071}, {'period': '2011', 'start_char': 1257, 'end_char': 1261}, {'period': 'July 2014', 'start_char': 1267, 'end_char': 1276}, {'period': 'November 2008-Apr 2011', 'start_char': 1998, 'end_char': 2020}, {'period': 'the financial year 2008-09', 'start_char': 2452, 'end_char': 2478}, {'period': 'June 2007', 'start_char': 2664, 'end_char': 2673}, {'period': 'October 2008', 'start_char': 2679, 'end_char': 2691}, {'period': 'May 2005', 'start_char': 3083, 'end_char': 3091}, {'period': 'May 2005', 'start_char': 3203, 'end_char': 3211}, {'period': 'Financial Year 2005-06', 'start_char': 3494, 'end_char': 35

#### Linking Company - Designation - Period

In [18]:
def get_experience(company_output, title_output, date_output):
    """
    Function to Link Company, Title & Period based on distances
    """
    out_ls = []
    if (len(company_output) > 0) and (len(title_output) > 0):
        for ls_1 in company_output:
            ls = []
            for ls_2 in title_output:
                _dict = {'company_name':ls_1['keyword'], 'title':ls_2['keyword'], 'distance':'', 
                         'company_start_idx':ls_1['start_idx'], 'company_end_idx':ls_1['end_idx']}
                company_end_idx = ls_1['end_idx']
                title_start_idx = ls_2['start_idx']
                _dict['distance'] = np.abs(company_end_idx-title_start_idx)
                ls.append(_dict)
            out_ls.append(min(ls, key=lambda x:x['distance']))
    
    full_ls = []
    
    if (len(out_ls) > 0) and (len(date_output) > 0):
        for ls_1 in out_ls:
            ls = []
            for ls_2 in date_output:
                _dict = {'company_name':ls_1['company_name'], 'title':ls_1['title'], 'date':ls_2['period'], 'distance_1':ls_1['distance'], 
                         'distance_2':''}
                company_end_idx = ls_1['company_end_idx']
                date_start_idx = ls_2['start_char']
                _dict['distance_2'] = np.abs(company_end_idx-date_start_idx)
                ls.append(_dict)
            full_ls.append(min(ls, key=lambda x:x['distance_2']))
    
    return full_ls

In [24]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print(">> FileName : ",file)
    print("\n")
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    company_output = get_company_names(text)
    title_output = get_job_titles(text)
    date_output = get_period(text)
    experience = get_experience(company_output, title_output, date_output)
    print(">> Companies: ", company_output)
    print("\n")
    print(">> Designation: ", title_output)
    print("\n")
    print(">> Experience: ", experience, end="\n---------------------------------------------- \n")

>> FileName :  Abha Doshi- CV.pdf


>> Companies:  [{'keyword': 'shah associates', 'start_idx': 8488, 'end_idx': 8503}, {'keyword': 'fine line circuits limited', 'start_idx': 8409, 'end_idx': 8435}, {'keyword': 'godrej group', 'start_idx': 5486, 'end_idx': 5498}, {'keyword': 'kpmg', 'start_idx': 3839, 'end_idx': 3843}, {'keyword': 'the hershey company', 'start_idx': 5525, 'end_idx': 5544}, {'keyword': 'godrej hershey', 'start_idx': 5363, 'end_idx': 5377}, {'keyword': 'ambit pragma advisors llp', 'start_idx': 1002, 'end_idx': 1027}]


>> Designation:  [{'keyword': 'vice president', 'start_idx': 6515, 'end_idx': 6529}, {'keyword': 'assistant manager', 'start_idx': 5624, 'end_idx': 5641}, {'keyword': 'senior consultant', 'start_idx': 423, 'end_idx': 440}, {'keyword': 'project manager', 'start_idx': 2711, 'end_idx': 2726}, {'keyword': 'finance professional', 'start_idx': 59, 'end_idx': 79}]


>> Experience:  [{'company_name': 'shah associates', 'title': 'vice president', 'date': 'May 2004'

>> Companies:  [{'keyword': 'standard chartered bank', 'start_idx': 5780, 'end_idx': 5803}, {'keyword': 'sbi mutual fund', 'start_idx': 2449, 'end_idx': 2464}, {'keyword': 'state bank of india', 'start_idx': 3752, 'end_idx': 3771}, {'keyword': 'sbi funds management', 'start_idx': 3376, 'end_idx': 3396}, {'keyword': 'mahindra special services group', 'start_idx': 106, 'end_idx': 137}, {'keyword': 'mm group', 'start_idx': 2523, 'end_idx': 2531}]


>> Designation:  [{'keyword': 'manager', 'start_idx': 3891, 'end_idx': 3898}, {'keyword': 'senior manager', 'start_idx': 3834, 'end_idx': 3848}, {'keyword': 'ceo', 'start_idx': 998, 'end_idx': 1001}]


>> Experience:  [{'company_name': 'standard chartered bank', 'title': 'manager', 'date': '2008', 'distance_1': 1912, 'distance_2': 11}, {'company_name': 'sbi mutual fund', 'title': 'senior manager', 'date': '2008-July 2015', 'distance_1': 1370, 'distance_2': 1063}, {'company_name': 'state bank of india', 'title': 'senior manager', 'date': 'Octobe

>> Companies:  [{'keyword': 'toll global logistics', 'start_idx': 1314, 'end_idx': 1335}, {'keyword': 'mahindra mahindra ltd', 'start_idx': 1605, 'end_idx': 1626}, {'keyword': 'tata steel', 'start_idx': 7558, 'end_idx': 7568}, {'keyword': 'allcargo logistics', 'start_idx': 1223, 'end_idx': 1241}, {'keyword': 'india value fund advisors', 'start_idx': 42, 'end_idx': 67}]


>> Designation:  [{'keyword': 'executive assistant', 'start_idx': 18536, 'end_idx': 18555}, {'keyword': 'associate', 'start_idx': 18339, 'end_idx': 18348}, {'keyword': 'financial management', 'start_idx': 1023, 'end_idx': 1043}, {'keyword': 'director', 'start_idx': 15191, 'end_idx': 15199}, {'keyword': 'chief executive', 'start_idx': 15149, 'end_idx': 15164}, {'keyword': 'manufacturing engineer', 'start_idx': 1740, 'end_idx': 1762}, {'keyword': 'manager', 'start_idx': 1507, 'end_idx': 1514}, {'keyword': 'managing director', 'start_idx': 1368, 'end_idx': 1385}, {'keyword': 'vice president', 'start_idx': 9293, 'end_idx':

>> Companies:  [{'keyword': 'abbott labs', 'start_idx': 6751, 'end_idx': 6762}, {'keyword': 'linde gases', 'start_idx': 6735, 'end_idx': 6746}, {'keyword': 'icici bank', 'start_idx': 4742, 'end_idx': 4752}, {'keyword': 'reliance capital ltd', 'start_idx': 6699, 'end_idx': 6719}, {'keyword': 'netambit', 'start_idx': 1323, 'end_idx': 1331}, {'keyword': 'blubirch', 'start_idx': 796, 'end_idx': 804}, {'keyword': 'knoll pharmaceuticals', 'start_idx': 5817, 'end_idx': 5838}, {'keyword': 'icici', 'start_idx': 5379, 'end_idx': 5384}, {'keyword': 'reliance capital', 'start_idx': 2458, 'end_idx': 2474}]


>> Designation:  [{'keyword': 'territory executive', 'start_idx': 5794, 'end_idx': 5813}, {'keyword': 'manager', 'start_idx': 6943, 'end_idx': 6950}, {'keyword': 'branch sales manager', 'start_idx': 4718, 'end_idx': 4738}, {'keyword': 'area sales manager', 'start_idx': 4677, 'end_idx': 4695}, {'keyword': 'regional sales manager', 'start_idx': 3907, 'end_idx': 3929}, {'keyword': 'sales manager',

>> Companies:  [{'keyword': 'tawam hospital', 'start_idx': 2078, 'end_idx': 2092}, {'keyword': 'dubai health authority', 'start_idx': 28596, 'end_idx': 28618}]


>> Designation:  [{'keyword': 'consultant', 'start_idx': 32939, 'end_idx': 32949}, {'keyword': 'registered nurse', 'start_idx': 4617, 'end_idx': 4633}, {'keyword': 'staff nurse', 'start_idx': 32366, 'end_idx': 32377}, {'keyword': 'supervisor', 'start_idx': 27284, 'end_idx': 27294}, {'keyword': 'nurse manager', 'start_idx': 31231, 'end_idx': 31244}, {'keyword': 'charge nurse', 'start_idx': 28812, 'end_idx': 28824}, {'keyword': 'nursing director', 'start_idx': 29024, 'end_idx': 29040}, {'keyword': 'medical practitioners', 'start_idx': 19450, 'end_idx': 19471}, {'keyword': 'supplier quality', 'start_idx': 18409, 'end_idx': 18425}, {'keyword': 'quality coordinator', 'start_idx': 9052, 'end_idx': 9071}, {'keyword': 'safety officer', 'start_idx': 9035, 'end_idx': 9049}, {'keyword': 'infection control coordinator', 'start_idx': 12645

...

>> Companies:  []


>> Designation:  [{'keyword': 'learning designer', 'start_idx': 5006, 'end_idx': 5023}, {'keyword': 'process owner', 'start_idx': 3262, 'end_idx': 3275}, {'keyword': 'management trainee', 'start_idx': 1179, 'end_idx': 1197}, {'keyword': 'manager', 'start_idx': 1162, 'end_idx': 1169}, {'keyword': 'hr manager', 'start_idx': 1145, 'end_idx': 1155}, {'keyword': 'general manager', 'start_idx': 1129, 'end_idx': 1144}, {'keyword': 'chairman', 'start_idx': 747, 'end_idx': 755}]


>> Experience:  []
---------------------------------------------- 
>> FileName :  Ujjal Bhattacharya- CV.pdf


>> Companies:  [{'keyword': 'mjunction services', 'start_idx': 45, 'end_idx': 63}, {'keyword': 'tata steel', 'start_idx': 919, 'end_idx': 929}, {'keyword': 'prime steel', 'start_idx': 418, 'end_idx': 429}]


>> Designation:  [{'keyword': 'vice president', 'start_idx': 19, 'end_idx': 33}, {'keyword': 'manager', 'start_idx': 876, 'end_idx': 883}, {'keyword': 'general manager', 'start_idx': 7

>> Companies:  [{'keyword': 'apollo hospitals', 'start_idx': 3371, 'end_idx': 3387}, {'keyword': 'aravind eye care', 'start_idx': 8673, 'end_idx': 8689}, {'keyword': 'sankara nethralaya', 'start_idx': 3054, 'end_idx': 3072}, {'keyword': 'kasturba hospital', 'start_idx': 6352, 'end_idx': 6369}, {'keyword': 'manipal group', 'start_idx': 5388, 'end_idx': 5401}, {'keyword': 'columbia asia', 'start_idx': 39, 'end_idx': 52}, {'keyword': 'apollo institute', 'start_idx': 3300, 'end_idx': 3316}]


>> Designation:  [{'keyword': 'management trainee', 'start_idx': 10156, 'end_idx': 10174}, {'keyword': 'internship', 'start_idx': 10120, 'end_idx': 10130}, {'keyword': 'internal auditor', 'start_idx': 3626, 'end_idx': 3642}, {'keyword': 'event manager', 'start_idx': 8106, 'end_idx': 8119}, {'keyword': 'manager', 'start_idx': 5456, 'end_idx': 5463}, {'keyword': 'assistant manager', 'start_idx': 5402, 'end_idx': 5419}, {'keyword': 'general manager', 'start_idx': 83, 'end_idx': 98}, {'keyword': 'procurem

>> Companies:  [{'keyword': 'oracle corp', 'start_idx': 54, 'end_idx': 65}, {'keyword': 'hyatt regency', 'start_idx': 19972, 'end_idx': 19985}, {'keyword': 'bristol myers squibb', 'start_idx': 13213, 'end_idx': 13233}, {'keyword': 'infosys', 'start_idx': 404, 'end_idx': 411}, {'keyword': 'infosys limited', 'start_idx': 4144, 'end_idx': 4159}, {'keyword': 'novel technologies', 'start_idx': 5835, 'end_idx': 5853}, {'keyword': 'between technology', 'start_idx': 1168, 'end_idx': 1186}]


>> Designation:  [{'keyword': 'intelligence applications', 'start_idx': 79, 'end_idx': 104}, {'keyword': 'data scientist', 'start_idx': 31, 'end_idx': 45}, {'keyword': 'senior principal', 'start_idx': 14, 'end_idx': 30}, {'keyword': 'staff engineer', 'start_idx': 9005, 'end_idx': 9019}, {'keyword': 'research assistant', 'start_idx': 11493, 'end_idx': 11511}, {'keyword': 'research intern', 'start_idx': 12847, 'end_idx': 12862}, {'keyword': 'software developer', 'start_idx': 12689, 'end_idx': 12707}, {'keywo

>> Companies:  [{'keyword': 'institute of computer', 'start_idx': 8071, 'end_idx': 8092}, {'keyword': 'computer software', 'start_idx': 8010, 'end_idx': 8027}, {'keyword': 'nisa group of companies', 'start_idx': 6022, 'end_idx': 6045}, {'keyword': 'bajaj electricals', 'start_idx': 5413, 'end_idx': 5430}, {'keyword': 'reliance retail', 'start_idx': 4906, 'end_idx': 4921}, {'keyword': 'panache globe solutions', 'start_idx': 193, 'end_idx': 216}]


>> Designation:  [{'keyword': 'manager', 'start_idx': 2578, 'end_idx': 2585}, {'keyword': 'business partner', 'start_idx': 4101, 'end_idx': 4117}, {'keyword': 'regional manager', 'start_idx': 4076, 'end_idx': 4092}, {'keyword': 'partner', 'start_idx': 2623, 'end_idx': 2630}, {'keyword': 'co founder', 'start_idx': 2254, 'end_idx': 2264}, {'keyword': 'director', 'start_idx': 225, 'end_idx': 233}, {'keyword': 'founder', 'start_idx': 217, 'end_idx': 224}]


>> Experience:  [{'company_name': 'institute of computer', 'title': 'business partner', 'dat

>> Companies:  [{'keyword': 'boston consulting group', 'start_idx': 4599, 'end_idx': 4622}, {'keyword': 'tata motors', 'start_idx': 4526, 'end_idx': 4537}, {'keyword': 'capital ltd', 'start_idx': 400, 'end_idx': 411}, {'keyword': 'tata motors finance', 'start_idx': 3960, 'end_idx': 3979}, {'keyword': 'icici bank', 'start_idx': 3056, 'end_idx': 3066}]


>> Designation:  [{'keyword': 'research scholar', 'start_idx': 5730, 'end_idx': 5746}, {'keyword': 'business representative', 'start_idx': 4550, 'end_idx': 4573}, {'keyword': 'visiting professor', 'start_idx': 4059, 'end_idx': 4077}, {'keyword': 'credit manager', 'start_idx': 3824, 'end_idx': 3838}, {'keyword': 'area sales manager', 'start_idx': 2678, 'end_idx': 2696}, {'keyword': 'sales manager', 'start_idx': 3152, 'end_idx': 3165}, {'keyword': 'senior manager', 'start_idx': 1889, 'end_idx': 1903}, {'keyword': 'avp', 'start_idx': 1714, 'end_idx': 1717}, {'keyword': 'organizational development specialist', 'start_idx': 1181, 'end_idx': 1

In [264]:
def get_full_experience(text, company_output, title_output, date_output):
    """
    Function to Link Company, Title & Period based on distances
    
    a) Input: Clean Text, Extracted Companies, Title & Dates
    
    b) Output: Mapped Company, Title & Dates
    """
    clean_txt = clean_special_text(text)
    
    out_ls = []
    if (len(company_output) > 0) and (len(title_output) > 0):
        for ls_1 in company_output:
            ls = []
            for ls_2 in title_output:
                _dict = {'company_name':ls_1['keyword'], 'title':ls_2['keyword'], 'distance':'', 
                         'company_start_idx':ls_1['start_idx'], 'company_end_idx':ls_1['end_idx'],
                         'title_start_idx':ls_2['start_idx'], 'title_end_idx':ls_2['end_idx']}
                company_end_idx = ls_1['end_idx']
                title_start_idx = ls_2['start_idx']
                _dict['distance'] = np.abs(company_end_idx-title_start_idx)
                if (company_end_idx > title_start_idx):
                    _dict['words'] = len(clean_txt[title_start_idx:company_end_idx].strip().split())
                else:
                    _dict['words'] = len(clean_txt[company_end_idx:title_start_idx].strip().split())

                if _dict['words'] > 50: ## If # of words between company & title exceed 50 then don't assign [Assumption]
                    _dict['title'] = ''
                ls.append(_dict)
            out_ls.append(min(ls, key=lambda x:x['distance']))

    full_ls = []

    if (len(out_ls) > 0) and (len(date_output) > 0):
        for ls_1 in out_ls:
            ls = []
            for ls_2 in date_output:
                _dict = {'company_name':ls_1['company_name'], 'title':ls_1['title'], 'date':ls_2['period'], 'distance_1':ls_1['distance'], 
                         'distance_2':'', 'words':ls_1['words']}
                company_end_idx = ls_1['company_end_idx']
                date_start_idx = ls_2['start_char']
                _dict['distance_2'] = np.abs(company_end_idx-date_start_idx)

                if (company_end_idx > date_start_idx):
                    _dict['date_words'] = len(clean_txt[date_start_idx:company_end_idx].strip().split())
                else:
                    _dict['date_words'] = len(clean_txt[company_end_idx:date_start_idx].strip().split())

                if _dict['date_words'] > 50: ## If # of words between company & date exceed 50 then don't assign [Assumption]
                    _dict['date'] = ''

                ls.append(_dict)
            full_ls.append(min(ls, key=lambda x:x['distance_2']))
    
    full_ls = sorted(full_ls, key = lambda i: (i['title'], i['words']), reverse=True)
    full_ls = list({i['company_name']:i for i in reversed(full_ls)}.values()) ## Remove dups
    return full_ls

In [265]:
file=os.listdir(RAW_DATA_PATH)[2]

text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
company_output = get_company_names(text)
title_output = get_job_titles(text)
date_output = get_period(text)
experience = get_full_experience(text, company_output, title_output, date_output)

ValueError: min() arg is an empty sequence

In [231]:
sorted(ls, key = lambda i: (i['title'], i['words']), reverse=True)

[{'company_name': 'flipkart',
  'title': 'xyz',
  'date': 'Oct 2013 - July 2014) - 10 months',
  'distance_1': 69,
  'distance_2': 22,
  'words': 0,
  'date_words': 3},
 {'company_name': 'flipkart',
  'title': 'senior product manager',
  'date': 'December 2012 - July 2014',
  'distance_1': 41,
  'distance_2': 64,
  'words': 2,
  'date_words': 11},
 {'company_name': 'interwoven inc',
  'title': '',
  'date': 'July 2005',
  'distance_1': 63,
  'distance_2': 191,
  'words': 1,
  'date_words': 31}]

In [230]:
ls=[{'company_name': 'interwoven inc',
  'title': '',
  'date': 'July 2005',
  'distance_1': 63,
  'distance_2': 191,
  'words': 1,
  'date_words': 31},
 {'company_name': 'flipkart',
  'title': 'senior product manager',
  'date': 'December 2012 - July 2014',
  'distance_1': 41,
  'distance_2': 64,
  'words': 2,
  'date_words': 11},
 {'company_name': 'flipkart',
  'title': 'xyz',
  'date': 'Oct 2013 - July 2014) - 10 months',
  'distance_1': 69,
  'distance_2': 22,
  'words': 0,
  'date_words': 3}]

In [223]:
sorted(ls, key = lambda i: i['words'], reverse=False)

[{'company_name': 'flipkart',
  'title': '',
  'date': 'Oct 2013 - July 2014) - 10 months',
  'distance_1': 69,
  'distance_2': 22,
  'words': 0,
  'date_words': 3},
 {'company_name': 'interwoven inc',
  'title': 'senior software developer',
  'date': 'July 2005',
  'distance_1': 63,
  'distance_2': 191,
  'words': 8,
  'date_words': 31},
 {'company_name': 'flipkart',
  'title': 'senior product manager',
  'date': 'December 2012 - July 2014',
  'distance_1': 41,
  'distance_2': 64,
  'words': 8,
  'date_words': 11}]

In [154]:
# clean_txt = clean_special_text(text)

In [169]:
# for txt in text.split("\n"):
#     i = DATE_KEYWORDS_SEARCH.extract_from_text(txt)
#     if len(i) > 0:
#         print(i)
#         matches = list(datefinder.find_dates(txt, index=True))
#         print(matches)
# #         if len(matches) > 0:
# #             print(" to ".join([str(k.date()) for k in matches]))
# #         elif len(matches)==1:
# #             print(str(matches[0].date()))

In [119]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print(">> FileName : ",file)
    print("\n")
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    company_output = get_company_names(text)
    title_output = get_job_titles(text)
    date_output = get_period(text)
    experience = get_full_experience(text, company_output, title_output, date_output)
    print(">> Companies: ", company_output)
    print("\n")
    print(">> Designation: ", title_output)
    print("\n")
    print(">> Experience: ", experience, end="\n---------------------------------------------- \n")

>> FileName :  Abha Doshi- CV.pdf


>> Companies:  [{'keyword': 'shah associates', 'start_idx': 8488, 'end_idx': 8503}, {'keyword': 'fine line circuits limited', 'start_idx': 8409, 'end_idx': 8435}, {'keyword': 'godrej group', 'start_idx': 5486, 'end_idx': 5498}, {'keyword': 'kpmg', 'start_idx': 3839, 'end_idx': 3843}, {'keyword': 'the hershey company', 'start_idx': 5525, 'end_idx': 5544}, {'keyword': 'godrej hershey', 'start_idx': 5363, 'end_idx': 5377}, {'keyword': 'ambit pragma advisors llp', 'start_idx': 1002, 'end_idx': 1027}]


>> Designation:  [{'keyword': 'vice president', 'start_idx': 6515, 'end_idx': 6529}, {'keyword': 'assistant manager', 'start_idx': 5624, 'end_idx': 5641}, {'keyword': 'senior consultant', 'start_idx': 423, 'end_idx': 440}, {'keyword': 'project manager', 'start_idx': 2711, 'end_idx': 2726}, {'keyword': 'finance professional', 'start_idx': 59, 'end_idx': 79}]


>> Experience:  [{'company_name': 'shah associates', 'title': '', 'date': '', 'distance_1': 1988, 

>> Companies:  [{'keyword': 'standard chartered bank', 'start_idx': 5780, 'end_idx': 5803}, {'keyword': 'sbi mutual fund', 'start_idx': 2449, 'end_idx': 2464}, {'keyword': 'state bank of india', 'start_idx': 3752, 'end_idx': 3771}, {'keyword': 'sbi funds management', 'start_idx': 3376, 'end_idx': 3396}, {'keyword': 'mahindra special services group', 'start_idx': 106, 'end_idx': 137}, {'keyword': 'mm group', 'start_idx': 2523, 'end_idx': 2531}]


>> Designation:  [{'keyword': 'manager', 'start_idx': 3891, 'end_idx': 3898}, {'keyword': 'senior manager', 'start_idx': 3834, 'end_idx': 3848}, {'keyword': 'ceo', 'start_idx': 998, 'end_idx': 1001}]


>> Experience:  [{'company_name': 'standard chartered bank', 'title': '', 'date': '2008', 'distance_1': 1912, 'distance_2': 11, 'words': 264, 'date_words': 2}, {'company_name': 'sbi mutual fund', 'title': '', 'date': '', 'distance_1': 1370, 'distance_2': 1063, 'words': 210, 'date_words': 156}, {'company_name': 'state bank of india', 'title': 'sen

>> Companies:  [{'keyword': 'the advertising standards council of india', 'start_idx': 5790, 'end_idx': 5832}, {'keyword': 'colgate palmolive', 'start_idx': 283, 'end_idx': 300}, {'keyword': 'ge money', 'start_idx': 3682, 'end_idx': 3690}, {'keyword': 'hindustan times', 'start_idx': 2861, 'end_idx': 2876}, {'keyword': 'ht media', 'start_idx': 337, 'end_idx': 345}, {'keyword': 'care media', 'start_idx': 1464, 'end_idx': 1474}, {'keyword': 'godrej consumer products', 'start_idx': 231, 'end_idx': 255}]


>> Designation:  [{'keyword': 'management trainee', 'start_idx': 5160, 'end_idx': 5178}, {'keyword': 'manager market research', 'start_idx': 4858, 'end_idx': 4881}, {'keyword': 'product manager', 'start_idx': 4363, 'end_idx': 4378}, {'keyword': 'general manager', 'start_idx': 3979, 'end_idx': 3994}, {'keyword': 'sales marketing', 'start_idx': 3619, 'end_idx': 3634}, {'keyword': 'associate vice president', 'start_idx': 3566, 'end_idx': 3590}, {'keyword': 'marketing lead', 'start_idx': 957,

>> Companies:  [{'keyword': 'abbott labs', 'start_idx': 6751, 'end_idx': 6762}, {'keyword': 'linde gases', 'start_idx': 6735, 'end_idx': 6746}, {'keyword': 'icici bank', 'start_idx': 4742, 'end_idx': 4752}, {'keyword': 'reliance capital ltd', 'start_idx': 6699, 'end_idx': 6719}, {'keyword': 'netambit', 'start_idx': 1323, 'end_idx': 1331}, {'keyword': 'blubirch', 'start_idx': 796, 'end_idx': 804}, {'keyword': 'knoll pharmaceuticals', 'start_idx': 5817, 'end_idx': 5838}, {'keyword': 'icici', 'start_idx': 5379, 'end_idx': 5384}, {'keyword': 'reliance capital', 'start_idx': 2458, 'end_idx': 2474}]


>> Designation:  [{'keyword': 'territory executive', 'start_idx': 5794, 'end_idx': 5813}, {'keyword': 'manager', 'start_idx': 6943, 'end_idx': 6950}, {'keyword': 'branch sales manager', 'start_idx': 4718, 'end_idx': 4738}, {'keyword': 'area sales manager', 'start_idx': 4677, 'end_idx': 4695}, {'keyword': 'regional sales manager', 'start_idx': 3907, 'end_idx': 3929}, {'keyword': 'sales manager',

>> Companies:  [{'keyword': 'coca cola', 'start_idx': 3554, 'end_idx': 3563}, {'keyword': 'deloitte', 'start_idx': 1618, 'end_idx': 1626}, {'keyword': 'national collateral management services limited', 'start_idx': 464, 'end_idx': 511}, {'keyword': 'kotak investment banking', 'start_idx': 125, 'end_idx': 149}]


>> Designation:  [{'keyword': 'president', 'start_idx': 3675, 'end_idx': 3684}, {'keyword': 'coo', 'start_idx': 3547, 'end_idx': 3550}, {'keyword': 'audit intern', 'start_idx': 1605, 'end_idx': 1617}, {'keyword': 'financial management', 'start_idx': 1569, 'end_idx': 1589}, {'keyword': 'teaching assistant', 'start_idx': 1459, 'end_idx': 1477}, {'keyword': 'analyst', 'start_idx': 117, 'end_idx': 124}]


>> Experience:  [{'company_name': 'coca cola', 'title': 'coo', 'date': '', 'distance_1': 16, 'distance_2': 349, 'words': 4, 'date_words': 48}, {'company_name': 'deloitte', 'title': 'audit intern', 'date': 'September November 2012', 'distance_1': 21, 'distance_2': 53, 'words': 3, '

...

>> Companies:  [{'keyword': 'apollo hospitals', 'start_idx': 3371, 'end_idx': 3387}, {'keyword': 'aravind eye care', 'start_idx': 8673, 'end_idx': 8689}, {'keyword': 'sankara nethralaya', 'start_idx': 3054, 'end_idx': 3072}, {'keyword': 'kasturba hospital', 'start_idx': 6352, 'end_idx': 6369}, {'keyword': 'manipal group', 'start_idx': 5388, 'end_idx': 5401}, {'keyword': 'columbia asia', 'start_idx': 39, 'end_idx': 52}, {'keyword': 'apollo institute', 'start_idx': 3300, 'end_idx': 3316}]


>> Designation:  [{'keyword': 'management trainee', 'start_idx': 10156, 'end_idx': 10174}, {'keyword': 'internship', 'start_idx': 10120, 'end_idx': 10130}, {'keyword': 'internal auditor', 'start_idx': 3626, 'end_idx': 3642}, {'keyword': 'event manager', 'start_idx': 8106, 'end_idx': 8119}, {'keyword': 'manager', 'start_idx': 5456, 'end_idx': 5463}, {'keyword': 'assistant manager', 'start_idx': 5402, 'end_idx': 5419}, {'keyword': 'general manager', 'start_idx': 83, 'end_idx': 98}, {'keyword': 'procurem

>> Companies:  [{'keyword': 'asian paints', 'start_idx': 3837, 'end_idx': 3849}, {'keyword': 'itc limited', 'start_idx': 3313, 'end_idx': 3324}, {'keyword': 'asian paints limited', 'start_idx': 1848, 'end_idx': 1868}, {'keyword': 'parag milk foods', 'start_idx': 1140, 'end_idx': 1156}, {'keyword': 'ikf finance', 'start_idx': 1069, 'end_idx': 1080}, {'keyword': 'motilal oswal', 'start_idx': 387, 'end_idx': 400}]


>> Designation:  [{'keyword': 'president', 'start_idx': 3587, 'end_idx': 3596}, {'keyword': 'senior manager', 'start_idx': 1887, 'end_idx': 1901}, {'keyword': 'board member', 'start_idx': 789, 'end_idx': 801}, {'keyword': 'vice president', 'start_idx': 416, 'end_idx': 430}, {'keyword': 'company secretary', 'start_idx': 145, 'end_idx': 162}, {'keyword': 'chartered accountant', 'start_idx': 121, 'end_idx': 141}]


>> Experience:  [{'company_name': 'asian paints', 'title': '', 'date': '2014', 'distance_1': 262, 'distance_2': 54, 'words': 37, 'date_words': 9}, {'company_name': 'it

>> Companies:  [{'keyword': 'fico', 'start_idx': 7813, 'end_idx': 7817}, {'keyword': 'kajaria ceramics', 'start_idx': 10360, 'end_idx': 10376}, {'keyword': 'varun beverages', 'start_idx': 10284, 'end_idx': 10299}, {'keyword': 'bank of america', 'start_idx': 2022, 'end_idx': 2037}, {'keyword': 'pizza hut', 'start_idx': 9048, 'end_idx': 9057}]


>> Designation:  [{'keyword': 'executive assistant', 'start_idx': 14936, 'end_idx': 14955}, {'keyword': 'engagement manager', 'start_idx': 14825, 'end_idx': 14843}, {'keyword': 'cfo', 'start_idx': 11593, 'end_idx': 11596}, {'keyword': 'project manager', 'start_idx': 1506, 'end_idx': 1521}, {'keyword': 'head of finance', 'start_idx': 12947, 'end_idx': 12962}, {'keyword': 'director', 'start_idx': 2328, 'end_idx': 2336}, {'keyword': 'vice president', 'start_idx': 11574, 'end_idx': 11588}, {'keyword': 'ceo', 'start_idx': 10975, 'end_idx': 10978}, {'keyword': 'people manager', 'start_idx': 10923, 'end_idx': 10937}, {'keyword': 'finance director', 'sta

>> Companies:  [{'keyword': 'tata teleservices', 'start_idx': 212, 'end_idx': 229}, {'keyword': 'jk organisation', 'start_idx': 8393, 'end_idx': 8408}, {'keyword': 'reliance industries limited', 'start_idx': 6615, 'end_idx': 6642}, {'keyword': 'virgin mobile', 'start_idx': 4681, 'end_idx': 4694}, {'keyword': 'tata teleservices limited', 'start_idx': 1423, 'end_idx': 1448}, {'keyword': 'thomas international', 'start_idx': 947, 'end_idx': 967}, {'keyword': 'the chemical company', 'start_idx': 1682, 'end_idx': 1702}, {'keyword': 'jk paper', 'start_idx': 1494, 'end_idx': 1502}]


>> Designation:  [{'keyword': 'chief executive', 'start_idx': 8569, 'end_idx': 8584}, {'keyword': 'sr manager', 'start_idx': 8554, 'end_idx': 8564}, {'keyword': 'partner', 'start_idx': 6951, 'end_idx': 6958}, {'keyword': 'vp', 'start_idx': 6763, 'end_idx': 6765}, {'keyword': 'manager', 'start_idx': 6740, 'end_idx': 6747}, {'keyword': 'project leader', 'start_idx': 6294, 'end_idx': 6308}, {'keyword': 'company manag

>> Companies:  [{'keyword': 'grant thornton', 'start_idx': 1709, 'end_idx': 1723}, {'keyword': 'mape advisory group', 'start_idx': 106, 'end_idx': 125}]


>> Designation:  [{'keyword': 'tax consultant', 'start_idx': 1957, 'end_idx': 1971}, {'keyword': 'internship', 'start_idx': 1324, 'end_idx': 1334}, {'keyword': 'analyst', 'start_idx': 126, 'end_idx': 133}]


>> Experience:  [{'company_name': 'grant thornton', 'title': '', 'date': '', 'distance_1': 234, 'distance_2': 554, 'words': 35, 'date_words': 91}, {'company_name': 'mape advisory group', 'title': 'analyst', 'date': '16 November, 1993', 'distance_1': 1, 'distance_2': 34, 'words': 0, 'date_words': 5}]
---------------------------------------------- 
>> FileName :  Yatin Joshi- CV.pdf


>> Companies:  [{'keyword': 'arabi company wll', 'start_idx': 7442, 'end_idx': 7459}, {'keyword': 'rdc concrete india', 'start_idx': 4195, 'end_idx': 4213}, {'keyword': 'fosroc chemicals india', 'start_idx': 2922, 'end_idx': 2944}, {'keyword': 'ambuja

### Education

#### College/School Extraction

In [193]:
file = "../2. Data/indian_colleges_list_v7.txt"
file = open(file, "r", encoding='utf-8')

COLLEGE_LS = [line.strip().lower() for line in file]

# college = list(set([clean_special_text(i) for i in college]))
# college = [i for i in college if len(i.split()) > 1]

# COLLEGE_LS = list(set(COLLEGE_LS) - set(['institute of technology', 'management studies', 'and technology', 'navi mumbai', 'b sc nursing', 'and college', 
# 'management research', 'college of technology and engineering']))

# with open('../2. Data/indian_colleges_list_v7.txt', 'w', encoding='utf-8') as f:
#     for item in COLLEGE_LS:
#         f.write("%s\n" % item)

In [194]:
## Initialize Keyword Processor:
COLLEGE_KEYWORDS_SEARCH = basic_keyword_extract(COLLEGE_LS, case_sensitive=False)

In [195]:
def get_college_names(text):
    text = clean_special_text(text)
    college_ls = COLLEGE_KEYWORDS_SEARCH.extract_from_text(text)
    return college_ls

In [21]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    college_output = get_college_names(text)    
    print("College: ", college_output, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
College:  [{'keyword': 'university of mumbai', 'start_idx': 7274, 'end_idx': 7294}, {'keyword': 'indian institute of management calcutta', 'start_idx': 7867, 'end_idx': 7906}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
College:  [{'keyword': 'nagpur university', 'start_idx': 4896, 'end_idx': 4913}, {'keyword': 'icai', 'start_idx': 4891, 'end_idx': 4895}, {'keyword': 'iim indore', 'start_idx': 693, 'end_idx': 703}]
---------------------------------------------- 
FileName :  Abhilash Pillai- CV.pdf
College:  [{'keyword': 'indian institute of management lucknow', 'start_idx': 4559, 'end_idx': 4597}, {'keyword': 'iim lucknow', 'start_idx': 1097, 'end_idx': 1108}]
---------------------------------------------- 
FileName :  Abhinaw Kumar- CV.pdf
College:  [{'keyword': 'indian institute of information technology allahabad', 'start_idx': 5706, 'end_idx': 5758}, {'keyword': 'indian institute of management indore', 'start_idx

College:  [{'keyword': 'bit mesra', 'start_idx': 2062, 'end_idx': 2071}, {'keyword': 'xavier institute of management india', 'start_idx': 1972, 'end_idx': 2008}]
---------------------------------------------- 
FileName :  Anshuman Mishra- CV.pdf
College:  [{'keyword': 'iit kharagpur', 'start_idx': 449, 'end_idx': 462}, {'keyword': 'iim lucknow', 'start_idx': 430, 'end_idx': 441}]
---------------------------------------------- 
FileName :  Anu Mishra- CV.pdf
College:  [{'keyword': 'iit kharagpur', 'start_idx': 6870, 'end_idx': 6883}]
---------------------------------------------- 
FileName :  Anuj Jain- CV.pdf
College:  [{'keyword': 'bit mesra', 'start_idx': 7363, 'end_idx': 7372}, {'keyword': 'birla institute of technology', 'start_idx': 6901, 'end_idx': 6930}, {'keyword': 'indian institute of management lucknow', 'start_idx': 6862, 'end_idx': 6900}]
---------------------------------------------- 
FileName :  Anuj Varma- CV.pdf
College:  [{'keyword': 'symbiosis college', 'start_idx': 6

College:  [{'keyword': 'institute of chartered accountants of india', 'start_idx': 645, 'end_idx': 688}, {'keyword': 'monjee college', 'start_idx': 405, 'end_idx': 419}]
---------------------------------------------- 
FileName :  Bishnu Sahu- CV.pdf
College:  [{'keyword': 'nit jamshedpur', 'start_idx': 804, 'end_idx': 818}, {'keyword': 'goa institute of management', 'start_idx': 740, 'end_idx': 767}]
---------------------------------------------- 
FileName :  Bobby Ramesh Kamath- CV (ISHWA).pdf
College:  [{'keyword': 'sikkim manipal university', 'start_idx': 11005, 'end_idx': 11030}, {'keyword': 'st thomas college of nursing', 'start_idx': 9261, 'end_idx': 9289}]
---------------------------------------------- 
FileName :  Chaitanya Shravanth- CV.pdf
College:  [{'keyword': 'army institute of technology', 'start_idx': 5885, 'end_idx': 5913}, {'keyword': 'iim ahmedabad', 'start_idx': 5760, 'end_idx': 5773}]
---------------------------------------------- 
FileName :  Chandrashekara Bhat P-

College:  [{'keyword': 'bangalore university', 'start_idx': 5296, 'end_idx': 5316}, {'keyword': 'bangalore medical college', 'start_idx': 5270, 'end_idx': 5295}]
---------------------------------------------- 
FileName :  Ezra Greenberg- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Feroz Zainulabdeen- CV.pdf
College:  [{'keyword': 'national institute of technology tiruchirappalli', 'start_idx': 894, 'end_idx': 942}]
---------------------------------------------- 
FileName :  Gajanan Nabar- CV.pdf
College:  [{'keyword': 'narsee monjee institute of management studies', 'start_idx': 1106, 'end_idx': 1151}]
---------------------------------------------- 
FileName :  Gaurav Khurana- CV (ISHWA).pdf
College:  [{'keyword': 'panjab university', 'start_idx': 11749, 'end_idx': 11766}]
---------------------------------------------- 
FileName :  GAURAV KOTHARI- CV.pdf
College:  [{'keyword': 'birla institute of technology and science pilani', 'start_idx': 3267, 'en

College:  [{'keyword': 'cfa institute', 'start_idx': 1292, 'end_idx': 1305}]
---------------------------------------------- 
FileName :  Kamal Bajaj- CV.pdf
College:  [{'keyword': 'delhi public school', 'start_idx': 579, 'end_idx': 598}]
---------------------------------------------- 
FileName :  KANISHKA CHAUDHARY- CV (Vito Talent Syndication).pdf
College:  [{'keyword': 'kolkata university', 'start_idx': 1232, 'end_idx': 1250}]
---------------------------------------------- 
FileName :  KANISHKA CHAUDHARY- CV (Vito).pdf
College:  [{'keyword': 'kolkata university', 'start_idx': 1232, 'end_idx': 1250}]
---------------------------------------------- 
FileName :  Kapil Hathi- CV.pdf
College:  [{'keyword': 'university of mumbai', 'start_idx': 1752, 'end_idx': 1772}, {'keyword': 'monjee college', 'start_idx': 1737, 'end_idx': 1751}]
---------------------------------------------- 
FileName :  KAPIL MODI- CV.pdf
College:  [{'keyword': 'iit kharagpur', 'start_idx': 3502, 'end_idx': 3515}, {'ke

College:  [{'keyword': 'university of mumbai', 'start_idx': 10142, 'end_idx': 10162}]
---------------------------------------------- 
FileName :  MITALI CHITRE- CV.pdf
College:  [{'keyword': 'mumbai university', 'start_idx': 6950, 'end_idx': 6967}, {'keyword': 'vidyalankar institute of technology', 'start_idx': 6914, 'end_idx': 6949}]
---------------------------------------------- 
FileName :  Monu Ratra- CV.pdf
College:  [{'keyword': 'dav college', 'start_idx': 2367, 'end_idx': 2378}, {'keyword': 'guru nanak dev university', 'start_idx': 2287, 'end_idx': 2312}, {'keyword': 'lal bahadur shastri institute of management', 'start_idx': 2120, 'end_idx': 2163}]
---------------------------------------------- 
FileName :  MR. KARAN BANSAL- CV.pdf
College:  [{'keyword': 'symbiosis college', 'start_idx': 5805, 'end_idx': 5822}]
---------------------------------------------- 
FileName :  Mr. Maruthi B.M- CV (EMA Partners International).pdf
College:  [{'keyword': 'central food technological resea

College:  [{'keyword': 'iit kanpur', 'start_idx': 106, 'end_idx': 116}, {'keyword': 'indian institute of technology', 'start_idx': 396, 'end_idx': 426}]
---------------------------------------------- 
FileName :  Piyush Kabra- CV (ISHWA).pdf
College:  [{'keyword': 'mumbai university', 'start_idx': 1300, 'end_idx': 1317}, {'keyword': 'icai', 'start_idx': 1250, 'end_idx': 1254}]
---------------------------------------------- 
FileName :  Piyush Nagori- CV (Micheal Page).pdf
College:  []
---------------------------------------------- 
FileName :  Prabhakar Sunder- CV.pdf
College:  [{'keyword': 'madras university', 'start_idx': 3995, 'end_idx': 4012}, {'keyword': 'institute of chartered accountants of india', 'start_idx': 3916, 'end_idx': 3959}]
---------------------------------------------- 
FileName :  PRADEEP JANARDHANAN- CV.pdf
College:  [{'keyword': 'shree kerala verma college thrissur', 'start_idx': 5132, 'end_idx': 5167}, {'keyword': 'business school new delhi', 'start_idx': 5064, '

College:  [{'keyword': 'cfa institute', 'start_idx': 1747, 'end_idx': 1760}, {'keyword': 'shri ram college of commerce', 'start_idx': 1683, 'end_idx': 1711}]
---------------------------------------------- 
FileName :  Rohan Nagpal- CV.pdf
College:  [{'keyword': 'delhi public school', 'start_idx': 3289, 'end_idx': 3308}]
---------------------------------------------- 
FileName :  Roopam Upadhyay- CV.pdf
College:  [{'keyword': 'national institute of industrial engineering', 'start_idx': 1087, 'end_idx': 1131}, {'keyword': 'indian institute of technology bombay', 'start_idx': 4139, 'end_idx': 4176}, {'keyword': 'iim lucknow', 'start_idx': 962, 'end_idx': 973}, {'keyword': 'iit bombay', 'start_idx': 291, 'end_idx': 301}]
---------------------------------------------- 
FileName :  Roselind Mathews- CV (ISHWA).pdf
College:  [{'keyword': 'shreemati nathibai damodar thackersey women s university', 'start_idx': 6992, 'end_idx': 7048}]
---------------------------------------------- 
FileName :  

College:  [{'keyword': 'university of mumbai', 'start_idx': 5492, 'end_idx': 5512}, {'keyword': 'indian school of business hyderabad', 'start_idx': 4378, 'end_idx': 4413}, {'keyword': 'indian institute of management ahmedabad', 'start_idx': 4229, 'end_idx': 4269}]
---------------------------------------------- 
FileName :  Shefali Bairaria Suri- CV.pdf
College:  [{'keyword': 'dav college', 'start_idx': 811, 'end_idx': 822}, {'keyword': 'symbiosis institute of management studies', 'start_idx': 750, 'end_idx': 791}]
---------------------------------------------- 
FileName :  Shekhar Tewari- CV.pdf
College:  [{'keyword': 'st josephs college', 'start_idx': 2015, 'end_idx': 2033}, {'keyword': 'sj college', 'start_idx': 1575, 'end_idx': 1585}]
---------------------------------------------- 
FileName :  Sheshadri Savalgi- CV.pdf
College:  [{'keyword': 'ms university baroda', 'start_idx': 6572, 'end_idx': 6592}, {'keyword': 'icai', 'start_idx': 6375, 'end_idx': 6379}]
-------------------------

College:  [{'keyword': 'college of engineering', 'start_idx': 2982, 'end_idx': 3004}, {'keyword': 'jai hind college', 'start_idx': 3047, 'end_idx': 3063}, {'keyword': 'mumbai university', 'start_idx': 3012, 'end_idx': 3029}]
---------------------------------------------- 
FileName :  TARANPREET SINGH- CV.pdf
College:  [{'keyword': 'amritsar college of engineering', 'start_idx': 5184, 'end_idx': 5215}, {'keyword': 'khalsa college amritsar', 'start_idx': 5231, 'end_idx': 5254}]
---------------------------------------------- 
FileName :  TEJAS A. RAVAL- CV.pdf
College:  [{'keyword': 'ms university baroda', 'start_idx': 6723, 'end_idx': 6743}, {'keyword': 'icai', 'start_idx': 6386, 'end_idx': 6390}]
---------------------------------------------- 
FileName :  Tejash Mahendra Tanna- CV.pdf
College:  [{'keyword': 'icai', 'start_idx': 6073, 'end_idx': 6077}]
---------------------------------------------- 
FileName :  Uddeeptabh Mishra- CV (Kelly Services).pdf
College:  [{'keyword': 'xavier ins

#### Degree Extraction

In [196]:
file = "../2. Data/degree_list_v3.txt"
file = open(file, "r", encoding='utf-8')

DEGREE_LS = [line.strip().lower() for line in file]

In [197]:
# DEGREE_LS = list(set([clean_special_text(i) for i in DEGREE_LS]))
# DEGREE_LS = [i for i in DEGREE_LS if len(i.split()) > 1]

# DEGREE_LS = list(set(DEGREE_LS) - set(['be']))

# with open('../2. Data/degree_list_v3.txt', 'w', encoding='utf-8') as f:
#     for item in DEGREE_LS:
#         f.write("%s\n" % item)

In [198]:
## Initialize Keyword Processor:
DEGREE_KEYWORDS_SEARCH = basic_keyword_extract(DEGREE_LS, case_sensitive=False)

In [199]:
def get_degree_names(text):
    text = clean_special_text(text)
    degree_ls = DEGREE_KEYWORDS_SEARCH.extract_from_text(text)
    return degree_ls

In [23]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    degree_output = get_degree_names(text)  
    print("Degree: ", degree_output, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Degree:  [{'keyword': 'mba', 'start_idx': 833, 'end_idx': 836}, {'keyword': 'accounting and finance', 'start_idx': 7345, 'end_idx': 7367}, {'keyword': 'bachelor of commerce', 'start_idx': 7324, 'end_idx': 7344}, {'keyword': 'master of commerce', 'start_idx': 7243, 'end_idx': 7261}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 4853, 'end_idx': 4858}, {'keyword': 'mba', 'start_idx': 681, 'end_idx': 684}, {'keyword': 'chartered accountant', 'start_idx': 122, 'end_idx': 142}]
---------------------------------------------- 
FileName :  Abhilash Pillai- CV.pdf
Degree:  [{'keyword': 'pgdm', 'start_idx': 5716, 'end_idx': 5720}]
---------------------------------------------- 
FileName :  Abhinaw Kumar- CV.pdf
Degree:  [{'keyword': 'b tech', 'start_idx': 5699, 'end_idx': 5705}, {'keyword': 'mba', 'start_idx': 5599, 'end_idx': 5602}, {'keyword': 'master of business administration', 'st

Degree:  [{'keyword': 'pgdm', 'start_idx': 6697, 'end_idx': 6701}]
---------------------------------------------- 
FileName :  Anuj Varma- CV.pdf
Degree:  [{'keyword': 'post graduate', 'start_idx': 7571, 'end_idx': 7584}, {'keyword': 'b com', 'start_idx': 6784, 'end_idx': 6789}, {'keyword': 'diploma', 'start_idx': 6622, 'end_idx': 6629}, {'keyword': 'mba', 'start_idx': 3305, 'end_idx': 3308}]
---------------------------------------------- 
FileName :  Anupam Jain- CV.pdf
Degree:  [{'keyword': 'mba', 'start_idx': 3523, 'end_idx': 3526}, {'keyword': 'pg', 'start_idx': 2153, 'end_idx': 2155}]
---------------------------------------------- 
FileName :  Anvaya Pratyush- CV.pdf
Degree:  [{'keyword': 'm tech', 'start_idx': 120, 'end_idx': 126}, {'keyword': 'b tech', 'start_idx': 109, 'end_idx': 115}]
---------------------------------------------- 
FileName :  APURVA RAI- CV.pdf
Degree:  [{'keyword': 'llb', 'start_idx': 72, 'end_idx': 75}, {'keyword': 'ba', 'start_idx': 69, 'end_idx': 71}]
---

Degree:  [{'keyword': 'b sc', 'start_idx': 477, 'end_idx': 481}]
---------------------------------------------- 
FileName :  Deepesh Purwani- CV (Kelly).pdf
Degree:  [{'keyword': 'b sc', 'start_idx': 477, 'end_idx': 481}]
---------------------------------------------- 
FileName :  Dhananjay Chithathoor- CV (ISHWA).pdf
Degree:  [{'keyword': 'bachelors in science', 'start_idx': 10805, 'end_idx': 10825}, {'keyword': 'pgdm', 'start_idx': 10735, 'end_idx': 10739}, {'keyword': 'mba', 'start_idx': 182, 'end_idx': 185}, {'keyword': 'bsc', 'start_idx': 178, 'end_idx': 181}]
---------------------------------------------- 
FileName :  Dhiraj Mangaramani- CV.pdf
Degree:  [{'keyword': 'bachelor of commerce', 'start_idx': 580, 'end_idx': 600}, {'keyword': 'chartered accountancy', 'start_idx': 493, 'end_idx': 514}]
---------------------------------------------- 
FileName :  Dilip Menda- CV (Ishwa).pdf
Degree:  [{'keyword': 'bachelor of engineering', 'start_idx': 7476, 'end_idx': 7499}, {'keyword': 'p

Degree:  [{'keyword': 'b tech', 'start_idx': 9730, 'end_idx': 9736}, {'keyword': 'mba', 'start_idx': 9613, 'end_idx': 9616}, {'keyword': 'master of business administration', 'start_idx': 9579, 'end_idx': 9612}]
---------------------------------------------- 
FileName :  Haresh Balani- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 3894, 'end_idx': 3899}, {'keyword': 'bachelor of commerce', 'start_idx': 3873, 'end_idx': 3893}, {'keyword': 'mba', 'start_idx': 3507, 'end_idx': 3510}]
---------------------------------------------- 
FileName :  Harsh A. Desai- CV.pdf
Degree:  [{'keyword': 'diploma', 'start_idx': 3389, 'end_idx': 3396}, {'keyword': 'bachelor of science in mechanical engineering', 'start_idx': 2998, 'end_idx': 3043}]
---------------------------------------------- 
FileName :  HARSH BUCHASIA- CV.pdf
Degree:  [{'keyword': 'bachelors in commerce', 'start_idx': 3211, 'end_idx': 3232}, {'keyword': 'company secretary', 'start_idx': 3130, 'end_idx': 3147}]
----------------------

Degree:  [{'keyword': 'chartered accountancy', 'start_idx': 2716, 'end_idx': 2737}, {'keyword': 'bachelor of commerce', 'start_idx': 2648, 'end_idx': 2668}, {'keyword': 'mba', 'start_idx': 2636, 'end_idx': 2639}]
---------------------------------------------- 
FileName :  Lakshay Kataria- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 2918, 'end_idx': 2923}, {'keyword': 'chartered accountancy', 'start_idx': 2857, 'end_idx': 2878}]
---------------------------------------------- 
FileName :  Laveesh Bhandari- CV.pdf
Degree:  [{'keyword': 'ba', 'start_idx': 2203, 'end_idx': 2205}, {'keyword': 'master of arts', 'start_idx': 2148, 'end_idx': 2162}, {'keyword': 'ph d', 'start_idx': 2105, 'end_idx': 2109}]
---------------------------------------------- 
FileName :  Laxman B D- CV (EMA Partners International).pdf
Degree:  [{'keyword': 'm sc', 'start_idx': 1579, 'end_idx': 1583}]
---------------------------------------------- 
FileName :  Linda Haskins- CV (Promed Health Recruitment).pdf
De

Degree:  [{'keyword': 'ba', 'start_idx': 19641, 'end_idx': 19643}, {'keyword': 'bachelor of arts', 'start_idx': 19624, 'end_idx': 19640}, {'keyword': 'post graduation', 'start_idx': 19539, 'end_idx': 19554}]
---------------------------------------------- 
FileName :  NEETU MAHESHWARI- CV.pdf
Degree:  []
---------------------------------------------- 
FileName :  Nelson DSouza- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 701, 'end_idx': 706}, {'keyword': 'chartered accountancy', 'start_idx': 622, 'end_idx': 643}]
---------------------------------------------- 
FileName :  Nelson Prabhakaran- CV (ISHWA).pdf
Degree:  [{'keyword': 'bms', 'start_idx': 4051, 'end_idx': 4054}, {'keyword': 'diploma', 'start_idx': 7847, 'end_idx': 7854}, {'keyword': 'b com', 'start_idx': 157, 'end_idx': 162}, {'keyword': 'mba', 'start_idx': 163, 'end_idx': 166}, {'keyword': 'pgdm', 'start_idx': 7617, 'end_idx': 7621}]
---------------------------------------------- 
FileName :  NIDHI KHADRIA- CV.pdf
Degre

Degree:  [{'keyword': 'b com', 'start_idx': 2828, 'end_idx': 2833}]
---------------------------------------------- 
FileName :  R. RAJASEKARAN- CV (EMA Partners International).pdf
Degree:  [{'keyword': 'm sc', 'start_idx': 3784, 'end_idx': 3788}, {'keyword': 'ph d', 'start_idx': 108, 'end_idx': 112}, {'keyword': 'bachelor of science in agriculture', 'start_idx': 4002, 'end_idx': 4036}, {'keyword': 'master of science', 'start_idx': 3668, 'end_idx': 3685}, {'keyword': 'doctor of philosophy', 'start_idx': 3195, 'end_idx': 3215}]
---------------------------------------------- 
FileName :  Radhika Kedia- CV.pdf
Degree:  [{'keyword': 'diploma', 'start_idx': 2800, 'end_idx': 2807}, {'keyword': 'b com', 'start_idx': 263, 'end_idx': 268}, {'keyword': 'chartered accountant', 'start_idx': 171, 'end_idx': 191}]
---------------------------------------------- 
FileName :  RAGHAV MANDHANA- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 464, 'end_idx': 469}, {'keyword': 'chartered accountant', 'st

Degree:  [{'keyword': 'mba', 'start_idx': 5519, 'end_idx': 5522}]
---------------------------------------------- 
FileName :  SANDEEP KHANDELWAL- CV.pdf
Degree:  [{'keyword': 'mba', 'start_idx': 6801, 'end_idx': 6804}]
---------------------------------------------- 
FileName :  SANDEEP RANJAN- CV.pdf
Degree:  [{'keyword': 'mba', 'start_idx': 6858, 'end_idx': 6861}, {'keyword': 'pgdm', 'start_idx': 6853, 'end_idx': 6857}]
---------------------------------------------- 
FileName :  Sanjay Gupta- CV.pdf
Degree:  [{'keyword': 'accounting and finance', 'start_idx': 4955, 'end_idx': 4977}, {'keyword': 'bcom', 'start_idx': 4950, 'end_idx': 4954}, {'keyword': 'bachelor of commerce', 'start_idx': 4929, 'end_idx': 4949}, {'keyword': 'llb', 'start_idx': 4827, 'end_idx': 4830}, {'keyword': 'bachelor of law', 'start_idx': 4811, 'end_idx': 4826}, {'keyword': 'chartered accountant', 'start_idx': 4707, 'end_idx': 4727}, {'keyword': 'company secretary', 'start_idx': 4635, 'end_idx': 4652}]
------------

Degree:  [{'keyword': 'bachelor of arts', 'start_idx': 9819, 'end_idx': 9835}, {'keyword': 'diploma', 'start_idx': 9628, 'end_idx': 9635}, {'keyword': 'post graduate', 'start_idx': 9614, 'end_idx': 9627}, {'keyword': 'advanced program in strategy for leaders', 'start_idx': 9503, 'end_idx': 9543}]
---------------------------------------------- 
FileName :  Snehamoy Mukherjee- CV.pdf
Degree:  [{'keyword': 'integrated msc', 'start_idx': 2675, 'end_idx': 2689}]
---------------------------------------------- 
FileName :  Snehamoy Mukhurjee- CV.pdf
Degree:  [{'keyword': 'integrated msc', 'start_idx': 2675, 'end_idx': 2689}]
---------------------------------------------- 
FileName :  Somesh Lal- CV (ISHWA).pdf
Degree:  [{'keyword': 'diploma', 'start_idx': 6399, 'end_idx': 6406}, {'keyword': 'bachelor of engineering', 'start_idx': 1395, 'end_idx': 1418}, {'keyword': 'mba', 'start_idx': 186, 'end_idx': 189}]
---------------------------------------------- 
FileName :  Sonal Varma- CV.pdf
Degree:

Degree:  [{'keyword': 'bachelors in business administration', 'start_idx': 943, 'end_idx': 979}, {'keyword': 'msc', 'start_idx': 249, 'end_idx': 252}]
---------------------------------------------- 
FileName :  Vijay Dhanuka- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 166, 'end_idx': 171}, {'keyword': 'company secretary', 'start_idx': 145, 'end_idx': 162}, {'keyword': 'chartered accountant', 'start_idx': 121, 'end_idx': 141}]
---------------------------------------------- 
FileName :  Vikas Agrawal- CV.pdf
Degree:  [{'keyword': 'ph d', 'start_idx': 15555, 'end_idx': 15559}, {'keyword': 'doctor of philosophy', 'start_idx': 15534, 'end_idx': 15554}]
---------------------------------------------- 
FileName :  Vikas Lal- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 993, 'end_idx': 998}, {'keyword': 'chartered accountant', 'start_idx': 933, 'end_idx': 953}]
---------------------------------------------- 
FileName :  Vikas Mittal- CV.pdf
Degree:  [{'keyword': 'chartered account

#### Linking College & Degree

In [260]:
def get_full_education(text, college_output, degree_output):
    """
    Function to Link College & Degree based on distances
    
    a) Input: Text, Extracted Colleges & Degree
    
    b) Output: Mapped Colleges, Degree
    """
    clean_txt = clean_special_text(text)
    
    out_ls = []
    if (len(college_output) > 0) and (len(degree_output) > 0):
        for ls_1 in college_output:
            ls = []
            for ls_2 in degree_output:
                _dict = {'college':ls_1['keyword'], 'degree':ls_2['keyword'], 'distance':'', 
                         'college_start_idx':ls_1['start_idx'], 'college_end_idx':ls_1['end_idx'],
                         'degree_start_idx':ls_2['start_idx'], 'degree_end_idx':ls_2['end_idx']}
                college_end_idx = ls_1['end_idx']
                degree_start_idx = ls_2['start_idx']
                _dict['distance'] = np.abs(college_end_idx-degree_start_idx)
                if (college_end_idx > degree_start_idx):
                    _dict['words'] = len(clean_txt[degree_start_idx:college_end_idx].strip().split())
                else:
                    _dict['words'] = len(clean_txt[college_end_idx:degree_start_idx].strip().split())

                if _dict['words'] > 50: ## If # of words between college & degree exceed 15 then don't assign [Assumption]
                    _dict['degree'] = ''
                ls.append(_dict)
            out_ls.append(min(ls, key=lambda x:x['distance']))
    
    out_ls = sorted(out_ls, key = lambda i: (i['degree'], i['words']), reverse=True)
    out_ls = list({i['college']:i for i in reversed(out_ls)}.values()) ## Remove dups
    return out_ls

In [266]:
# file=os.listdir(RAW_DATA_PATH)[0]

# text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
# college_output = get_college_names(text)    
# degree_output = get_degree_names(text)  
# education = get_full_education(text, college_output, degree_output)

In [201]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print(">> FileName : ",file)
    print("\n")
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    college_output = get_college_names(text)    
    degree_output = get_degree_names(text)  
    education = get_full_education(text, college_output, degree_output)
    print(">> College: ", college_output)
    print("\n")
    print(">> Degree: ", degree_output)
    print("\n")
    print(">> Education: ", education, end="\n---------------------------------------------- \n")

>> FileName :  Abha Doshi- CV.pdf


>> College:  [{'keyword': 'university of mumbai', 'start_idx': 7274, 'end_idx': 7294}, {'keyword': 'university of mumbai', 'start_idx': 7368, 'end_idx': 7388}, {'keyword': 'indian institute of management calcutta', 'start_idx': 7867, 'end_idx': 7906}, {'keyword': 'university of mumbai', 'start_idx': 8169, 'end_idx': 8189}]


>> Degree:  [{'keyword': 'mba', 'start_idx': 833, 'end_idx': 836}, {'keyword': 'mba', 'start_idx': 6925, 'end_idx': 6928}, {'keyword': 'master of commerce', 'start_idx': 7243, 'end_idx': 7261}, {'keyword': 'bachelor of commerce', 'start_idx': 7324, 'end_idx': 7344}, {'keyword': 'accounting and finance', 'start_idx': 7345, 'end_idx': 7367}, {'keyword': 'mba', 'start_idx': 7962, 'end_idx': 7965}]


>> Education:  [{'college': 'university of mumbai', 'degree': 'bachelor of commerce', 'distance': 30, 'college_start_idx': 7274, 'college_end_idx': 7294, 'degree_start_idx': 7324, 'degree_end_idx': 7344, 'words': 4}, {'college': 'indian 

>> College:  [{'keyword': 'the oberoi centre of learning and development', 'start_idx': 5143, 'end_idx': 5188}, {'keyword': 'canan school of catering and hotel management', 'start_idx': 5209, 'end_idx': 5254}]


>> Degree:  [{'keyword': 'post graduate diploma in hospitality management', 'start_idx': 5273, 'end_idx': 5320}, {'keyword': 'diploma in hospitality management', 'start_idx': 5321, 'end_idx': 5354}]


>> Education:  [{'college': 'canan school of catering and hotel management', 'degree': 'post graduate diploma in hospitality management', 'distance': 19, 'college_start_idx': 5209, 'college_end_idx': 5254, 'degree_start_idx': 5273, 'degree_end_idx': 5320, 'words': 3}, {'college': 'the oberoi centre of learning and development', 'degree': 'post graduate diploma in hospitality management', 'distance': 85, 'college_start_idx': 5143, 'college_end_idx': 5188, 'degree_start_idx': 5273, 'degree_end_idx': 5320, 'words': 14}]
---------------------------------------------- 
>> FileName :  A

>> College:  [{'keyword': 'indian institute of technology bombay', 'start_idx': 2704, 'end_idx': 2741}, {'keyword': 'iit bombay', 'start_idx': 2808, 'end_idx': 2818}, {'keyword': 'college of engineering', 'start_idx': 3112, 'end_idx': 3134}]


>> Degree:  [{'keyword': 'bachelor of technology', 'start_idx': 2742, 'end_idx': 2764}]


>> Education:  [{'college': 'college of engineering', 'degree': '', 'distance': 392, 'college_start_idx': 3112, 'college_end_idx': 3134, 'degree_start_idx': 2742, 'degree_end_idx': 2764, 'words': 59}, {'college': 'iit bombay', 'degree': 'bachelor of technology', 'distance': 76, 'college_start_idx': 2808, 'college_end_idx': 2818, 'degree_start_idx': 2742, 'degree_end_idx': 2764, 'words': 11}, {'college': 'indian institute of technology bombay', 'degree': 'bachelor of technology', 'distance': 1, 'college_start_idx': 2704, 'college_end_idx': 2741, 'degree_start_idx': 2742, 'degree_end_idx': 2764, 'words': 0}]
---------------------------------------------- 
>> F

>> College:  [{'keyword': 'royal college', 'start_idx': 1960, 'end_idx': 1973}]


>> Degree:  [{'keyword': 'diploma', 'start_idx': 1346, 'end_idx': 1353}, {'keyword': 'diploma', 'start_idx': 1605, 'end_idx': 1612}]


>> Education:  [{'college': 'royal college', 'degree': '', 'distance': 368, 'college_start_idx': 1960, 'college_end_idx': 1973, 'degree_start_idx': 1605, 'degree_end_idx': 1612, 'words': 51}]
---------------------------------------------- 
>> FileName :  Anja Stodtmeister- CV (Promed Heath Recruitment).pdf




KeyboardInterrupt: 

### Skills

In [42]:
file = "../2. Data/skills_clean_list_v2.txt"
file = open(file, "r", encoding='utf-8')

SKILL_LS = [line.strip().lower() for line in file]

In [43]:
## Initialize Keyword Processor:
SKILLS_KEYWORDS_SEARCH = basic_keyword_extract(SKILL_LS, case_sensitive=False)

In [44]:
def extract_skills(text):
    text = clean_special_text(text)
    skill_ls = SKILLS_KEYWORDS_SEARCH.extract_from_text(text)
    return skill_ls

In [46]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    skills_output = extract_skills(text)    
    print("Skills: ", list(set([i['keyword'].upper() for i in skills_output])), end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Skills:  ['INTERNATIONAL MANAGEMENT', 'HINDI', 'ONLINE PRESENCE', 'DUTCH', 'BUSINESS OPERATIONS', 'REPORTING', 'BUILDING', 'FINANCE', 'INVESTMENT', 'PAY', 'PROJECT', 'BUDGETING FORECASTING', 'MID MARKET', 'OPERATIONS', 'LIABILITY', 'COLD CHAIN', 'FINANCIAL ADVISORY', 'ACCOUNTANTS', 'VOLUNTARY', 'LED', 'FINANCIAL STATEMENTS', 'DUE DILIGENCE', 'TEASERS', 'EAGLE', 'SYNERGIES', 'GOLD', 'FRANCHISE', 'DEAL SOURCING', 'CONTROL ENVIRONMENT', 'TEAMS', 'ECONOMICS', 'INVESTMENTS', 'GENERAL', 'CIRCUITS', 'ORDER TO CASH', 'DOCUMENTATION', 'PET FOOD', 'COST EFFECTIVE', 'IFRS', 'FINANCIAL REPORTING', 'ASSESSMENT', 'CXO', 'DIAMONDS', 'MEASURES', 'ROAMBI', 'PRODUCT MIX', 'BOOKKEEPING', 'GUJARATI', 'INTERNAL CONTROLS', 'CLOSURE', 'FLEET', 'COMPONENTS', 'OPERATIONAL DUE DILIGENCE', 'DECISION MAKING', 'CLOUD', 'COORDINATED', 'LANGUAGES', 'ACADEMIC BACKGROUND', 'FIXED ASSETS', 'GROWTH CAPITAL', 'STUDIO', 'AUTOMOTIVE', 'GROSS MARGIN', 'CORPORATE FINANCE', 'INTERNAL AUDIT', 'FI

Skills:  ['COMPENSATION AND BENEFITS', 'HINDI', 'EMPLOYEE RELATIONS', 'PROGRAMMES', 'POLICIES PROCEDURES', 'REPORTING', 'COMPENSATION', 'BUDGETING', 'COOKING', 'PIPELINE', 'PAY', 'PERFORMANCE MANAGEMENT', 'MAN POWER', 'RECORD', 'PROJECT', 'COMPENSATION BENEFITS', 'MUTUAL FUNDS', 'TECHNICAL ADVISORY', 'BANDING', 'DESIGNATION', 'TVC', 'MMS', 'COMPENSATION BENCHMARKING', 'SECURITIES', 'CAPITAL MARKETS', 'NEGOTIATION', 'MARATHON', 'PAYROLL', 'BENCHMARKING', 'FINANCIAL SERVICES', 'STAFF TRAINING', 'HR TRANSFORMATION', 'PERSONNEL FILES', 'BUNGALOWS', 'VIZ', 'LEARNING', 'SOURCING', 'REASON', 'INTERVIEWING', 'FRAUD', 'ESIC', 'BARRIERS', 'MAIL', 'BIRTH', 'ASSET MANAGEMENT', 'BOARDING', 'RESEARCH', 'LANGUAGES', 'FORMULATION', 'CAMPUS', 'CHEMISTRY', 'REGULATIONS', 'WEB TECHNOLOGIES', 'GOVERNMENT', 'INVESTORS', 'LEGAL REQUIREMENTS', 'GRIEVANCES', 'OPERATIONAL TRANSFORMATION', 'HOBBIES', 'REVISIONS', 'ROAD', 'SURVEY', 'INFORMATION SECURITY', 'SALARY', 'INTELLIGENCE', 'RISK', 'RISK CONSULTING', 'ONB

Skills:  ['MARKET RESEARCH', 'WWW', 'AGENDA', 'RETAIL AUDIT', 'CIRCULATION', 'BRAND EQUITY', 'STRATEGIC THINKING', 'PROMO', 'CONSUMER PRODUCTS', 'MULTI COUNTRY', 'CONSUMER FOCUS', 'MARKETING LEADERSHIP', 'BUILDING', 'RESEARCH', 'PRODUCT INNOVATION', 'GENERAL ELECTRIC', 'MEDIA RESEARCH', 'SUPPLY CHAIN', 'NIELSEN', 'CAMPAIGNS', 'INSECTICIDES', 'MONEY', 'BRAND STRATEGY', 'RANGE', 'EMAIL', 'FINANCE', 'LAUNCHES', 'MAR', 'INSURANCE', 'FINISHED GOODS', 'MARKETING MIX', 'MEDIA MARKETING', 'PRODUCT QUALITY', 'APAC', 'PRODUCTION PLANNING', 'NET', 'GLOBAL SALES', 'PERSONAL CARE', 'TEAMS', 'DETERGENTS', 'ELEMENTS', 'DIGITAL', 'PROJECT', 'GENERAL', 'RECORD', 'DEMAND', 'ANALYTICS', 'DIARY', 'NEWSPAPERS', 'DIGITAL MARKETING', 'LEADERSHIP', 'FMCG', 'SOLUTION SELLING', 'HOUSEHOLD', 'PROFITABLE GROWTH', 'LED', 'MARKET SHARE']
---------------------------------------------- 
FileName :  Ajit Jangle- CV (HEDRICKS & STRUGGLES).pdf
Skills:  ['SALES PERFORMANCE', 'REACH', 'STRATEGIC THINKING', 'REPORTING', 'D

Skills:  ['COMPLEXITY REDUCTION', 'DATABASES', 'REPORTING', 'RANGE', 'BUDGETING', 'EDUCATION SOFTWARE', 'PHYSICS', 'FINANCE', 'PROGRAM MANAGEMENT', 'PHARMACEUTICALS', 'INVESTMENT', 'RESPONSE MODELING', 'RAN', 'FRENCH', 'MULTIPLES', 'OPERATING BUDGETS', 'DIRECTORS', 'PROJECT', 'RESOURCE ALLOCATION', 'OPERATIONS', 'LIABILITY', 'PRICING STRATEGY', 'LEADERSHIP', 'FMCG', 'CAPITAL', 'BLOOMBERG', 'AUTOMATION', 'LED', 'COST EFFICIENCY', 'EBITDA', 'DIRECT MARKETING', 'COACHING', 'LAKE', 'PROCESS MANAGEMENT', 'BUSINESS ANALYSIS', 'ROIC', 'HAIR CARE', 'LANDSCAPING', 'CAPACITY UTILIZATION', 'GOLD', 'LAUNCHES', 'STRUCTURAL', 'TEXAS', 'INVESTMENT BANKING', 'MATLAB', 'PERSONAL CARE', 'TEAMS', 'PERFORMANCE ANALYSIS', 'TRIALS', 'INVESTMENTS', 'REMODELING', 'SKIN CARE', 'AVG', 'VALUE CHAIN ANALYSIS', 'SPSS', 'ASSESSMENT', 'SHARED SERVICES', 'ORBIT', 'AUDIT COMMITTEE', 'FRAMEWORK', 'INTERNAL CONTROLS', 'OWNERSHIP', 'PREDICTION', 'BUSINESS PLANNING', 'TAX', 'SAVINGS', 'SUPPLY CHAIN', 'CORPORATE BUDGETING'

Skills:  ['WATER', 'ACT', 'STRATEGIC PLANNING', 'NRP', 'TESTING', 'BOX', 'NURSERY', 'INTENSIVE CARE', 'VIP', 'PUBLIC HEALTH', 'POLICIES PROCEDURES', 'CONSCIOUS SEDATION', 'HISTORY', 'DELEGATION', 'ASSISTANTS', 'GENERATORS', 'SECURITY', 'UROLOGY', 'FERTILITY', 'STAFF DEVELOPMENT', 'FINALIZATION', 'INTERPRETING', 'PACKING', 'POSITIVE WORK ENVIRONMENT', 'DASHBOARDS', 'ADMINISTRATIVE', 'PREVENTION', 'POLLUTION', 'FIRE TRAINING', 'TEACHING', 'RENOVATION', 'NEPHROLOGY', 'PROFESSIONAL DEVELOPMENT', 'MAN', 'INJURY', 'CONSUMABLES', 'ORGANIZATIONAL ETHICS', 'PATIENT EDUCATION', 'THEATERS', 'MICROSOFT WORD', 'RADIATION', 'CONSTRUCTION', 'FACILITATION', 'BARIATRICS', 'PERFORMANCE MANAGEMENT', 'SKYPE', 'RECORD', 'PROCEDURAL', 'REGISTRATION', 'LIFE SUPPORT', 'LAUNDRY', 'PROCEDURE MANUALS', 'MEDICAL EQUIPMENT', 'PRESSURE', 'FLOOD', 'ENERGY', 'CARDIOLOGY', 'PREPARATION', 'PROFESSIONAL ASSOCIATIONS', 'ACTING', 'LEADERSHIP', 'CAPITAL', 'AVAILABILITY', 'SPECIFICATIONS', 'RECORDS', 'DIALYSIS', 'HEALTH SCI

...

Skills:  ['UNION', 'COACHING', 'HINDI', 'INCOME TAX', 'OFFICERS', 'SAP ERP', 'ANIMAL HEALTH', 'RETAIL', 'DOCUMENTARIES', 'ELECTRONICS', 'COMMERCE', 'PROVISION', 'TAX PLANNING', 'BUILDING', 'FINANCIAL SYSTEMS', 'REPORTING', 'CHESS', 'TAX', 'EBITDA', 'LIQUIDITY', 'HEALTH', 'BUSINESS PLANNING', 'GERMAN', 'FINANCIAL STRATEGY', 'SAVINGS', 'LANGUAGES', 'HEALTHCARE', 'FINANCIAL STATEMENTS', 'USER TRAINING', 'BUDGETING', 'FIXED ASSETS', 'TRANSFER PRICING', 'CISA', 'SEM', 'FINANCE', 'STRUCTURAL', 'DIRECT TAX', 'VENDOR MANAGEMENT', 'LEARNING', 'BRANCH BANKING', 'BUSINESS PROCESS IMPROVEMENT', 'GROSS MARGIN', 'INVENTORY CONTROL', 'MANUFACTURING', 'TAX AUDITS', 'PLANTS', 'SDL', 'ORION', 'TAX DEDUCTED AT SOURCE TDS', 'INTERNAL AUDIT', 'PAPER', 'ITALIAN', 'COST REDUCTION', 'INFORMATION TECHNOLOGY', 'ISACA', 'BUDGETING FORECASTING', 'INCENTIVES', 'OPERATIONS', 'STRATEGY', 'BUSINESS UNITS', 'ENGLISH', 'STATUTORY AUDIT', 'IFRS', 'PREPARATION', 'ACCOUNT CREATION', 'SAP IMPLEMENTATION', 'VENDORS', 'RBI',

Skills:  ['LEVEL III', 'SHAPE', 'PUBLICATIONS', 'INSTITUTIONAL', 'MATERIALS', 'SENSITIVITY', 'TIER 2', 'MEDICARE', 'BENCHMARKING', 'MANAGEMENT CONSULTING', 'COMPANY VALUATION', 'TELECOM', 'TELECOMMUNICATIONS', 'SERVICE PROVIDERS', 'RESEARCH', 'OEMS', 'SILVER', 'ASIAN MARKETS', 'LAL', 'PERSONAL BANKING', 'CABLE', 'SMALL CAP', 'DEVELOPERS', 'TRADING DESK', 'TECHNOLOGY', 'SPACE', 'HOCKEY', 'INVESTMENT', 'FRANCHISE', 'INVESTORS', 'BUSINESS CASE', 'PIPELINE', 'SCENARIO ANALYSIS', 'SEGMENTATION', 'TRADING', 'ACCESS', 'HOSPITALS', 'APOLLO', 'WRITING', 'TEAMS', 'EVENTS', 'MULTIPLES', 'AUGMENTED REALITY', 'MEDIA TECHNOLOGY', 'GENERAL', 'STRATEGY', 'MARKET ENTRY', 'MODELS', 'EQUITIES', 'CONFERENCES', 'ENERGY', 'INVESTMENTS', 'STOCKS', 'SPECTRUM', 'BRONZE', 'BOUTIQUE', 'UNIVERSITIES', 'LEADERSHIP', 'DSL', 'FINANCIAL RISK', 'REVISIONS', 'ROAD', 'DDM', 'ASSESSMENT', 'WEB', 'CONSUMER ELECTRONICS', 'DCF', 'RATINGS', 'CASE STUDIES', 'HEDGE FUNDS', 'ADDITION', 'LOCATION BASED SERVICES', 'FRAMEWORK', 'C

Skills:  ['ACT', 'STRATEGIC PLANNING', 'REACH', 'YAHOO', 'INCOME TAX', 'CHINESE', 'API', 'INDICATORS', 'REPORTING', 'APPEALS', 'BUILDING', 'CONTINUOUS IMPROVEMENT', 'FINALIZATION', 'BUDGETING', 'FINANCE', 'PHARMACEUTICALS', 'TEACHING', 'FINANCIAL PERFORMANCE', 'INVESTMENT', 'DIRECT TAX', 'QRM', 'CAREER DEVELOPMENT', 'EXIST', 'PROJECT', 'OPERATIONS', 'MECHANISMS', 'REAL ESTATE', 'ENERGY', 'PREPARATION', 'FRAMING', 'LEADERSHIP', 'CALCULATIONS', 'CASH FLOW', 'TRIBUNALS', 'CAPITAL', 'ACCOUNTANTS', 'EMBEDDED', 'SALES OPERATIONS', 'EBITDA', 'COACHING', 'PERFORMANCE REPORTING', 'ROLLING', 'FINANCIALS', 'COST MANAGEMENT', 'DUE DILIGENCE', 'BENCHMARKING', 'LEGAL ISSUES', 'FINANCIAL OPERATIONS', 'SALES GROWTH', 'MONEY', 'GARDEN', 'HUMILITY', 'MITIGATION', 'PURCHASING', 'GENERAL MOTORS', 'CONTROL ENVIRONMENT', 'CUSTOMER PROFITABILITY', 'APPROVALS', 'TEAMS', 'GENERAL', 'DOCUMENTATION', 'BUSINESS PROFITABILITY', 'IFRS', 'DIVIDENDS', 'ASSESSMENT', 'VALIDATION', 'CREDIT', 'BIRTH', 'POWERTRAIN', 'FRAM

Skills:  ['INCREASE PRODUCTIVITY', 'STRATEGIC PLANNING', 'BUSINESS PROCESS', 'FINALIZATION', 'INTERPRETING', 'BUDGETING', 'DASHBOARDS', 'INVOICING', 'PROBABILITY', 'CHART', 'FINANCE', 'FINANCIAL PERFORMANCE', 'FOREIGN CURRENCY', 'INVESTMENT', 'PIPELINE', 'DRAWING', 'DEVELOPMENT CENTERS', 'LOANS', 'RECORD', 'PROJECT', 'RESOURCE ALLOCATION', 'OPEX', 'OPERATIONS', 'PROCESS VALIDATION', 'UTILIZATION REVIEW', 'SG A', 'PRICING STRATEGY', 'PREPARATION', 'LEADERSHIP', 'IRR', 'CASH FLOW', 'CAPITAL', 'REVIEWS', 'ACCOUNTANTS', 'LONG RANGE PLANNING', 'MACRO', 'FINANCIAL STATEMENTS', 'KEY PERFORMANCE INDICATORS', 'FINANCIALS', 'LOAN', 'NUTS', 'BENCHMARKING', 'NEW STORE OPENINGS', 'LAUNCHES', 'STRUCTURAL', 'VIZ', 'FLASH', 'LEARNING', 'LIQUIDATION', 'FINANCIAL DATA', 'PROCESS EFFICIENCY', 'TEAM MANAGEMENT', 'TEAMS', 'GENERAL', 'FINANCIAL REPORTING', 'MEASURES', 'CREDIT', 'SUBSIDIARIES', 'BIRTH', 'MANAGEMENT PROFESSIONAL', 'FRAMEWORK', 'ADHERENCE', 'PROMPT', 'WORKING CAPITAL MANAGEMENT', 'STORE', 'PEO

Skills:  ['POLYMER COMPOUNDING', 'DUE DILIGENCE', 'ART', 'MANDATES', 'SELL SIDE', 'HEALTHCARE ANALYTICS', 'TRA', 'BUILDING', 'SME', 'RESEARCH', 'TAX', 'REIKI', 'COMPLETION', 'POLYPROPYLENE', 'CRYPTO', 'DCF VALUATION', 'INVESTORS', 'COMPOUNDING', 'SCENARIO ANALYSIS', 'PROCESS ENGINEERING', 'CURRENCY', 'MODELS', 'BASE', 'NBFC', 'LIABILITY', 'FACILITIES MANAGEMENT', 'VALUATION', 'CLASSIFICATION', 'PACKAGING', 'BIRTH', 'CREDIT RATING', 'FOUNDATION', 'RUBBER']
---------------------------------------------- 
FileName :  Yatin Joshi- CV.pdf
Skills:  ['WATER', 'WLL', 'CREDIT CONTROL', 'MATERIALS', 'SEALANTS', 'COMMUNICATIONS', 'NOTES', 'TIME MANAGEMENT', 'READY MIX CONCRETE', 'MOULD', 'APPLICATION', 'TUNNELING', 'PROTECTION', 'FLOORING', 'RESEARCH', 'BUILDING', 'MULTINATIONAL', 'TROUBLESHOOTING', 'RANGE', 'STRUCTURAL REPAIRS', 'AGGREGATES', 'PROTECTIVE COATINGS', 'RAPID GROWTH', 'CONSTRUCTION', 'RDC', 'PRINT MEDIA', 'PAPER', 'TEAM MANAGEMENT', 'PRECAST', 'PRODUCT LAUNCH', 'CONCRETE', 'SALES MA

### Table Extraction

In [275]:
file_path = r"C:\Users\admin\Downloads\nic_2008_17apr09.pdf"

In [276]:
import camelot

In [303]:
page_list = ",".join([str(i) for i in np.arange(36,138)])

In [305]:
table_list = camelot.read_pdf(file_path, 
                              pages=page_list, 
                              flavor="stream", 
                              flag_size=True,
                              line_size_scaling=80)

In [307]:
tab_df = pd.DataFrame()

In [308]:
tab_ls = []

In [309]:
for i in range(len(table_list)):
    tab_df = pd.concat([tab_df, table_list[i].df],
                      axis=0,
                      ignore_index=True).drop_duplicates()

In [355]:
tab_df.columns = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5']

In [360]:
df = tab_df[["col_3", "col_4"]]

In [361]:
df = df.replace(r'^\s*$', np.nan, regex=True)

In [363]:
df.to_clipboard()

In [333]:
df["col_3"] = df["col_3"].ffill()

In [336]:
df = df.dropna()

In [340]:
df

,col_3,col_4
2,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",relateD ServiCe aCtivitieS
3,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",Growing of non-perennial crops
4,"DiviSion 01 : Crop anD animal proDuCtion, hunt...","Growing of cereals (except rice), leguminous c..."
5,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",This class includes all forms of growing of ce...
6,"DiviSion 01 : Crop anD animal proDuCtion, hunt...","oil seeds in open fields, including those cons..."
...,...,...
4294,98200,activities of extraterritorial organizations a...
4295,98200,This class includes the activities of internat...
4296,98200,"United Nations and its agencies, IMF, World Ba..."
4297,98200,OPEC etc.


In [347]:
df

,col_3,col_4
2,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",relateD ServiCe aCtivitieS
3,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",Growing of non-perennial crops
4,"DiviSion 01 : Crop anD animal proDuCtion, hunt...","Growing of cereals (except rice), leguminous c..."
5,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",This class includes all forms of growing of ce...
6,"DiviSion 01 : Crop anD animal proDuCtion, hunt...","oil seeds in open fields, including those cons..."
...,...,...
4294,98200,activities of extraterritorial organizations a...
4295,98200,This class includes the activities of internat...
4296,98200,"United Nations and its agencies, IMF, World Ba..."
4297,98200,OPEC etc.


In [348]:
for i in df.index:
    df.loc[i, "flag"] = df.loc[i, "col_3"].isnumeric()

In [351]:
main_df = df[df['flag']==True]

In [354]:
main_df.groupby(['col_3'], as_index=False).agg(lambda x: " ".join(x)).to_clipboard()

In [335]:
df.to_clipboard()

In [323]:
df["col_3"].ffill()

0                                                        
1       DiviSion 01 : Crop anD animal proDuCtion, hunt...
2                                                        
3                                                        
4                                                        
                              ...                        
4294                                                     
4295                                                     
4296                                                     
4297                                                     
4298                                                99000
Name: col_3, Length: 4295, dtype: object

In [311]:
tab_df.to_clipboard()